In [1]:
using Pkg
using Distances
using StatsBase
using DataStructures
using ProgressMeter
using JSON
Pkg.activate("TopicModels")
import TopicModels
using PyCall
using PyPlot

using LSHFunctions, LinearAlgebra, BenchmarkTools
#corpora = pyimport("gensim.corpora")
#ch = pyimport("gensim.models.coherencemodel")

 Activating environment at `~/Documents/Study/Git_Personal/julia_bayes/TopicModels/Project.toml`


In [2]:
#nn = pyimport("torch.nn")
np = pyimport("numpy")
torch = pyimport("torch")
#plt = pyimport("matplotlib.pyplot")
f1_score = pyimport("sklearn.metrics")
feature_selection = pyimport("sklearn.feature_selection")

PyObject <module 'sklearn.feature_selection' from '/Users/khan/.julia/conda/3/lib/python3.8/site-packages/sklearn/feature_selection/__init__.py'>

In [3]:
#using 10,000 docs is too much becasue kp_similarity file exceeds 65GB, 5,000 is high so now using 2,000
corpus, no_lemma_docs = TopicModels.preprocess("20news/20newsgroup_long_only.csv", 2000); #Took around 1:25 mimnutes

In [4]:
#load lda object from saved json
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json"); #only takes few seconds 

#or train new lda model from below command, uncomment to run.
#lda = TopicModels.train(corpus, 20); #training took around 15 mints
#TopicModels.saveLDA(lda, "20news/lda_obj_2000_long.json");

In [5]:
topics, proportions = TopicModels.show_topics(lda, corpus, 10);

Any["god", "jesus", "bible", "church", "christ", "christian", "christians", "faith", "lord", "believe"]
----------------------
Any["said", "people", "one", "know", "would", "going", "came", "children", "went", "like"]
----------------------
Any["evidence", "one", "book", "world", "two", "many", "history", "life", "years", "etc"]
----------------------
Any["said", "one", "police", "city", "times", "years", "days", "kill", "soldiers", "two"]
----------------------
Any["armenian", "turkish", "armenians", "jews", "war", "armenia", "russian", "azerbaijan", "genocide", "turkey"]
----------------------
Any["game", "year", "good", "car", "get", "last", "got", "games", "first", "players"]
----------------------
Any["image", "jpeg", "file", "gif", "quality", "color", "use", "format", "version", "may"]
----------------------
Any["would", "one", "like", "people", "think", "know", "even", "make", "way", "good"]
----------------------
Any["health", "medical", "cancer", "disease", "patients", "april"

In [9]:
topic_distributions = TopicModels.sortedTopDocsForTopics(lda, corpus);

In [10]:
top_topic_for_each_doc = TopicModels.topic_of_each_doc(lda, corpus); # function implementation in #add_kp section
#println(top_topic_for_each_doc)

In [11]:
topic_number = 1
topic_x_top_docs = [idx for (idx, val) in enumerate(top_topic_for_each_doc) if val==topic_number];
size(topic_x_top_docs)

(225,)

In [108]:
println(topic_x_top_docs)

[8, 18, 19, 23, 27, 43, 44, 67, 69, 84, 86, 87, 88, 101, 113, 116, 130, 145, 190, 222, 223, 226, 232, 235, 249, 256, 267, 292, 293, 294, 321, 332, 338, 339, 341, 346, 371, 393, 413, 417, 425, 433, 434, 435, 469, 478, 494, 495, 499, 519, 521, 529, 535, 548, 559, 560, 570, 606, 622, 625, 631, 639, 648, 661, 662, 671, 679, 680, 715, 717, 727, 731, 740, 753, 759, 772, 774, 776, 779, 799, 804, 806, 828, 833, 848, 867, 882, 900, 914, 921, 960, 966, 970, 972, 980, 982, 1013, 1035, 1036, 1059, 1062, 1063, 1067, 1074, 1080, 1106, 1120, 1130, 1136, 1137, 1143, 1147, 1155, 1157, 1161, 1169, 1187, 1188, 1194, 1196, 1208, 1216, 1222, 1233, 1240, 1260, 1264, 1274, 1276, 1278, 1282, 1285, 1297, 1298, 1310, 1326, 1329, 1333, 1350, 1352, 1359, 1377, 1379, 1380, 1393, 1394, 1398, 1400, 1401, 1412, 1414, 1427, 1430, 1432, 1434, 1435, 1442, 1448, 1472, 1475, 1480, 1487, 1517, 1522, 1527, 1538, 1550, 1552, 1561, 1565, 1567, 1572, 1584, 1625, 1638, 1642, 1662, 1674, 1677, 1678, 1692, 1693, 1705, 1709, 1714,

In [12]:
fileKPDoc = "20news/kp/20news_doc_keyphrases_long_only_clean_last_2000.json"
fileEMB = "20news/kp/20news_keyphrase_long_only_2000_embedding.json"
fileSim = "20news/kp/20news_keyphrase_long_only_2000_similarity.json"
fileKP = "20news/kp/20news_keyphrases_long_only_2000.json"
kp = TopicModels.load_keyphrase(fileKPDoc, fileEMB, fileSim, fileKP);

In [13]:
cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, topic_distributions, 10, 1, 0.75);

In [14]:
x = 3
println(sort(cluster_kp[x]))
print(sort(docs_have[x]))

OrderedDict{Any,Any}("posit" => Any["posit", "decrypt"])
Any[249, 717, 753, 970, 1062, 1074, 1169, 1240]

In [302]:
TopicModels.removeDoc(lda, corpus, docs_have[3], 1) #remove posit kp from topic 1
TopicModels.gibbsSampling(lda, corpus.documents, 20);

In [304]:
topic_number = 1
top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
#println(top_topic_for_each_doc)
topic_x_top_docs = [idx for (idx, val) in enumerate(top_topic_for_each_doc) if val==topic_number];
size(topic_x_top_docs)

(207,)

In [306]:
print(topic_x_top_docs)

[8, 18, 19, 23, 27, 43, 44, 67, 84, 86, 87, 88, 101, 108, 113, 116, 130, 145, 190, 222, 223, 226, 232, 235, 256, 267, 292, 293, 294, 321, 332, 338, 339, 341, 346, 368, 371, 393, 413, 417, 433, 434, 435, 469, 478, 494, 495, 519, 521, 529, 535, 548, 559, 560, 606, 622, 625, 631, 639, 648, 662, 671, 679, 680, 708, 715, 727, 731, 740, 759, 772, 774, 776, 779, 799, 806, 828, 833, 848, 866, 867, 882, 890, 900, 914, 921, 960, 966, 972, 980, 982, 1013, 1036, 1059, 1067, 1080, 1106, 1120, 1130, 1136, 1137, 1147, 1155, 1157, 1161, 1187, 1188, 1194, 1196, 1208, 1216, 1222, 1233, 1260, 1274, 1276, 1282, 1285, 1293, 1297, 1298, 1310, 1326, 1329, 1333, 1350, 1352, 1359, 1377, 1379, 1380, 1393, 1394, 1398, 1400, 1401, 1412, 1414, 1427, 1430, 1434, 1435, 1442, 1446, 1448, 1472, 1475, 1480, 1487, 1517, 1522, 1527, 1538, 1550, 1561, 1565, 1567, 1584, 1625, 1638, 1642, 1662, 1674, 1677, 1678, 1692, 1693, 1705, 1709, 1714, 1716, 1739, 1749, 1753, 1763, 1767, 1774, 1779, 1788, 1792, 1824, 1836, 1839, 1840,

In [31]:
#This kind of implementation is not a good idea(conclusion after checking the association numbers)
cluster_kp_all_docs, docs_have_from_all_docs = TopicModels.top_x_kp_of_topic_m_for_all_docs(kp, topic_distributions, 10, 1, 0.8);


## Remove_word refinement after stable marriage algorithm

In [9]:
true_cat_original = [17, 7, 18, 17, 10, 19, 16, 19, 18, 8, 13, 7, 12, 9, 18, 18, 17, 17, 15, 17, 7, 1, 19, 5, 4, 8, 19, 17, 15, 15, 17, 0, 19, 11, 17, 18, 9, 13, 19, 17, 13, 17, 19, 15, 7, 8, 11, 9, 1, 8, 0, 18, 17, 14, 14, 10, 15, 3, 8, 10, 10, 11, 17, 6, 17, 17, 15, 8, 15, 11, 2, 17, 9, 18, 17, 17, 10, 2, 9, 1, 10, 4, 9, 15, 3, 15, 19, 15, 16, 13, 17, 11, 0, 18, 10, 14, 6, 17, 15, 17, 19, 14, 8, 14, 5, 5, 11, 15, 6, 10, 5, 9, 15, 11, 9, 15, 7, 1, 14, 16, 15, 13, 13, 17, 0, 11, 16, 3, 16, 15, 14, 14, 19, 9, 18, 14, 17, 17, 4, 17, 15, 1, 16, 11, 15, 6, 2, 14, 15, 18, 8, 17, 18, 4, 7, 11, 17, 18, 9, 16, 18, 7, 11, 2, 15, 19, 1, 11, 14, 17, 1, 16, 6, 5, 19, 7, 11, 16, 8, 16, 10, 15, 17, 16, 16, 1, 0, 7, 15, 0, 6, 18, 10, 9, 11, 16, 19, 14, 6, 0, 0, 3, 17, 17, 12, 16, 8, 6, 19, 10, 17, 16, 14, 4, 16, 11, 1, 14, 0, 2, 16, 15, 15, 6, 15, 0, 14, 10, 15, 13, 2, 19, 15, 17, 15, 1, 9, 16, 18, 17, 6, 17, 8, 14, 11, 5, 15, 7, 15, 15, 12, 11, 14, 14, 3, 15, 3, 17, 3, 14, 11, 19, 10, 18, 15, 16, 15, 15, 9, 16, 11, 4, 19, 13, 3, 19, 9, 17, 18, 7, 7, 15, 16, 14, 15, 10, 5, 9, 14, 14, 17, 15, 15, 19, 14, 15, 15, 0, 4, 11, 11, 15, 12, 6, 3, 15, 14, 5, 14, 18, 11, 11, 14, 17, 17, 10, 9, 5, 7, 10, 19, 18, 17, 15, 18, 17, 11, 11, 13, 3, 2, 15, 10, 17, 2, 14, 13, 15, 15, 16, 15, 13, 14, 7, 16, 19, 3, 17, 17, 17, 14, 16, 9, 8, 14, 14, 16, 13, 12, 11, 17, 11, 1, 4, 11, 18, 16, 15, 11, 17, 15, 6, 17, 17, 13, 9, 10, 18, 17, 17, 13, 6, 1, 9, 14, 19, 3, 19, 13, 14, 10, 16, 19, 14, 9, 17, 7, 7, 11, 17, 18, 11, 17, 11, 18, 17, 16, 14, 1, 11, 16, 11, 18, 6, 1, 14, 15, 6, 17, 18, 16, 6, 13, 11, 0, 0, 16, 8, 17, 17, 10, 4, 15, 15, 15, 1, 13, 15, 14, 5, 15, 9, 1, 0, 13, 11, 1, 14, 16, 7, 2, 6, 10, 8, 14, 12, 19, 2, 7, 15, 17, 13, 17, 13, 17, 17, 19, 16, 15, 0, 12, 19, 19, 18, 15, 16, 9, 15, 16, 16, 12, 9, 9, 18, 4, 18, 11, 17, 15, 12, 12, 17, 17, 15, 19, 15, 19, 17, 15, 10, 14, 19, 16, 17, 10, 10, 14, 16, 5, 13, 9, 15, 17, 15, 19, 0, 18, 14, 15, 1, 15, 5, 17, 12, 18, 10, 5, 16, 0, 15, 1, 5, 7, 11, 19, 13, 11, 17, 9, 18, 9, 4, 10, 11, 5, 15, 12, 15, 1, 18, 6, 13, 18, 9, 1, 17, 18, 17, 15, 15, 19, 3, 17, 15, 9, 11, 14, 17, 14, 15, 1, 15, 15, 10, 3, 19, 17, 14, 6, 11, 12, 15, 17, 15, 13, 3, 0, 10, 4, 11, 0, 14, 7, 0, 15, 0, 2, 17, 17, 6, 19, 14, 17, 14, 6, 15, 12, 13, 4, 1, 17, 16, 17, 13, 18, 0, 11, 10, 3, 0, 13, 15, 13, 15, 15, 13, 5, 18, 1, 11, 19, 15, 13, 13, 10, 17, 13, 14, 19, 14, 0, 5, 7, 15, 11, 1, 15, 15, 14, 15, 13, 0, 15, 18, 7, 8, 13, 11, 16, 14, 19, 15, 16, 0, 4, 17, 11, 10, 11, 5, 19, 10, 17, 7, 10, 17, 9, 11, 15, 19, 17, 11, 11, 11, 10, 16, 14, 17, 18, 13, 16, 19, 9, 11, 18, 16, 1, 13, 16, 18, 16, 11, 16, 8, 14, 14, 16, 15, 13, 15, 11, 11, 16, 2, 15, 16, 19, 8, 17, 19, 0, 16, 15, 14, 17, 9, 19, 6, 10, 16, 15, 0, 17, 10, 6, 16, 16, 17, 11, 15, 14, 10, 2, 18, 0, 18, 16, 10, 8, 3, 17, 17, 15, 11, 17, 13, 9, 18, 19, 11, 17, 11, 11, 5, 18, 17, 13, 11, 14, 13, 17, 15, 2, 0, 12, 0, 0, 14, 15, 8, 18, 9, 17, 10, 15, 13, 13, 3, 4, 16, 10, 5, 2, 2, 16, 17, 15, 7, 15, 17, 16, 12, 17, 15, 16, 15, 17, 17, 17, 18, 19, 17, 14, 18, 11, 12, 18, 6, 14, 11, 0, 15, 10, 17, 3, 11, 9, 15, 11, 9, 13, 16, 15, 13, 17, 18, 10, 4, 18, 19, 5, 19, 17, 11, 10, 19, 17, 19, 18, 10, 3, 13, 10, 16, 11, 5, 0, 3, 11, 17, 13, 14, 6, 16, 4, 19, 19, 14, 1, 14, 16, 11, 6, 18, 19, 11, 10, 11, 19, 11, 9, 19, 10, 17, 10, 18, 14, 11, 9, 0, 9, 1, 15, 11, 8, 14, 18, 11, 16, 15, 16, 6, 18, 13, 18, 2, 17, 15, 16, 15, 11, 16, 5, 15, 11, 14, 1, 16, 15, 11, 19, 16, 11, 15, 17, 15, 2, 16, 11, 7, 13, 17, 16, 7, 10, 1, 16, 1, 18, 17, 11, 17, 11, 5, 17, 1, 18, 0, 18, 16, 17, 12, 17, 11, 11, 15, 15, 12, 0, 19, 10, 17, 14, 12, 0, 19, 11, 2, 16, 19, 17, 15, 10, 5, 16, 5, 5, 13, 11, 19, 4, 15, 14, 11, 17, 15, 14, 8, 17, 3, 13, 0, 17, 0, 10, 12, 14, 6, 13, 13, 11, 10, 13, 10, 14, 0, 19, 11, 7, 10, 14, 3, 15, 3, 14, 10, 17, 4, 5, 11, 0, 19, 10, 10, 4, 1, 13, 17, 7, 5, 19, 13, 11, 14, 19, 19, 18, 2, 13, 6, 9, 0, 11, 11, 6, 11, 18, 16, 14, 10, 2, 8, 16, 5, 17, 16, 13, 18, 17, 13, 18, 15, 19, 6, 18, 17, 19, 12, 18, 9, 13, 6, 0, 15, 11, 18, 11, 0, 18, 15, 10, 10, 6, 5, 17, 11, 1, 9, 1, 14, 12, 14, 10, 15, 11, 11, 18, 2, 10, 3, 17, 18, 14, 19, 17, 15, 9, 5, 1, 9, 13, 15, 13, 10, 17, 19, 4, 17, 17, 15, 10, 17, 13, 18, 17, 11, 18, 18, 15, 15, 17, 11, 10, 8, 0, 15, 15, 14, 11, 15, 11, 8, 15, 3, 12, 8, 15, 14, 15, 10, 10, 11, 11, 3, 15, 9, 19, 17, 4, 7, 15, 4, 9, 9, 1, 6, 13, 10, 19, 9, 11, 16, 19, 17, 16, 14, 10, 9, 15, 15, 9, 5, 0, 2, 18, 9, 15, 19, 19, 17, 5, 10, 14, 15, 17, 15, 18, 17, 15, 4, 8, 13, 8, 11, 1, 15, 13, 15, 9, 9, 17, 14, 10, 14, 0, 15, 0, 17, 10, 0, 17, 0, 10, 14, 0, 12, 16, 17, 13, 13, 19, 3, 15, 0, 16, 19, 11, 14, 17, 19, 0, 14, 2, 18, 17, 15, 2, 17, 3, 11, 14, 9, 4, 3, 9, 0, 11, 13, 5, 15, 10, 14, 10, 15, 0, 9, 17, 13, 0, 5, 17, 16, 7, 0, 19, 15, 18, 15, 17, 0, 1, 15, 11, 17, 15, 9, 18, 6, 11, 17, 12, 13, 15, 18, 7, 17, 15, 15, 6, 13, 11, 18, 4, 5, 13, 7, 17, 18, 17, 15, 18, 0, 6, 5, 18, 13, 14, 2, 19, 8, 8, 10, 8, 17, 11, 15, 10, 17, 0, 19, 8, 6, 15, 12, 15, 16, 12, 19, 6, 6, 9, 14, 14, 17, 16, 16, 13, 6, 13, 15, 6, 15, 0, 10, 10, 15, 13, 14, 15, 17, 10, 15, 17, 11, 5, 9, 12, 16, 10, 13, 16, 2, 17, 6, 5, 0, 15, 9, 15, 19, 0, 11, 18, 1, 3, 18, 18, 18, 17, 0, 4, 17, 15, 15, 17, 1, 1, 15, 1, 15, 15, 16, 19, 17, 15, 2, 17, 15, 11, 16, 11, 15, 12, 15, 6, 13, 16, 19, 3, 5, 5, 7, 6, 1, 1, 16, 15, 18, 0, 15, 0, 15, 17, 19, 15, 0, 16, 14, 17, 6, 7, 15, 11, 12, 15, 15, 17, 15, 12, 17, 1, 1, 14, 3, 6, 17, 17, 18, 15, 1, 6, 10, 18, 10, 2, 11, 13, 17, 16, 3, 17, 19, 19, 18, 15, 8, 5, 16, 11, 0, 9, 13, 0, 9, 10, 1, 15, 1, 10, 17, 1, 14, 16, 1, 14, 15, 11, 14, 6, 8, 15, 9, 19, 16, 7, 17, 14, 17, 15, 18, 5, 11, 15, 2, 16, 5, 15, 9, 12, 17, 10, 15, 17, 19, 9, 1, 15, 17, 18, 3, 16, 18, 1, 13, 3, 18, 18, 15, 17, 17, 9, 19, 3, 11, 16, 17, 10, 1, 7, 19, 9, 19, 1, 17, 10, 18, 17, 8, 10, 17, 19, 13, 5, 18, 15, 11, 19, 0, 14, 11, 15, 15, 18, 18, 16, 13, 2, 16, 12, 5, 6, 0, 5, 15, 3, 16, 8, 8, 1, 17, 13, 18, 17, 17, 14, 6, 16, 14, 4, 3, 3, 3, 17, 7, 4, 14, 15, 18, 11, 16, 17, 15, 17, 6, 9, 13, 17, 6, 17, 18, 3, 17, 15, 3, 15, 18, 10, 17, 2, 18, 11, 13, 11, 15, 1, 10, 18, 19, 14, 11, 18, 15, 0, 1, 15, 17, 15, 8, 11, 18, 1, 7, 19, 10, 0, 9, 17, 9, 17, 11, 1, 19, 7, 15, 13, 17, 17, 0, 9, 16, 18, 18, 13, 15, 5, 7, 15, 15, 4, 15, 5, 5, 8, 9, 11, 13, 12, 16, 17, 17, 1, 19, 15, 14, 13, 11, 13, 13, 13, 14, 2, 9, 15, 7, 15, 2, 8, 11, 15, 11, 0, 4, 11, 15, 0, 15, 1, 5, 17, 5, 11, 5, 4, 8, 17, 3, 5, 14, 5, 13, 10, 16, 17, 9, 7, 6, 10, 10, 19, 3, 14, 17, 13, 1, 16, 0, 7, 17, 15, 0, 11, 15, 19, 16, 18, 0, 10, 17, 1, 17, 1, 15, 15, 9, 12, 14, 15, 11, 10, 11, 16, 13, 17, 15, 2, 13, 19, 19, 15, 16, 10, 19, 6, 14, 17, 13, 19, 15, 6, 17, 16, 15, 17, 18, 7, 12, 14, 17, 13, 1, 5, 9, 0, 6, 4, 17, 4, 13, 17, 16, 19, 1, 6, 14, 11, 11, 9, 14, 10, 16, 3, 12, 17, 15, 17, 1, 3, 13, 14, 10, 5, 14, 15, 6, 10, 15, 16, 14, 15, 15, 9, 1, 17, 15, 19, 17, 2, 9, 9, 1, 15, 17, 17, 15, 16, 16, 14, 9, 17, 17, 10, 4, 15, 17, 16, 18, 18, 19, 17, 3, 12, 7, 11, 3, 13, 12, 15, 1, 19, 17, 12, 16, 9, 8, 14, 0, 16, 5, 10, 15, 15, 15, 12, 17, 18, 15, 14, 2, 15, 18, 0, 15, 4, 1, 1, 1, 13, 13, 3, 16, 9, 19, 17, 12, 10, 18, 0, 0, 14, 7, 13, 14, 17, 13, 17, 1, 6, 9, 15, 4, 15, 8, 17, 18, 7, 19, 16, 9, 6, 16, 10, 14, 6, 18, 18, 11, 17, 7, 18, 13, 3, 17, 17, 15, 19, 19, 11, 0, 17, 15, 19, 18, 15, 5, 10, 0, 0, 0, 1, 13, 0, 12, 18, 8, 0, 2, 15, 17, 18, 6, 17, 13, 15, 0, 17, 18, 15, 18, 13, 19, 10, 8, 15, 2, 1, 15, 17, 15, 17, 17]
docs_cat = [i+1 for i in true_cat_original];
docs_ls = [[] for i in 1:20]
for (idx, val) in enumerate(docs_cat)
    push!(docs_ls[val], idx)
end

In [10]:
topic_marriage_assigment = Dict(1=>16, 2=>9, 3=>20, 4=>5, 5=>18, 6=>10, 7=>2, 8=>1, 9=>13, 10=>4, 
                            11=>7, 12=>12, 13=>8, 14=>17, 15=>6, 16=>14, 17=>19, 18=>15, 19=>11, 20=>3);

In [233]:
already_reomved_words = [];

In [253]:
#First loop refinement for all topics
for current_cat in 1:20
    false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
    remaining = [i for i in 1:corpus.document_size if !(i in docs_ls[current_cat]) && !(i in false_docs_for_cat)]
    if false_docs_for_cat != []
        X, Y = make_PMI_dataset(false_docs_for_cat, remaining)
        word_idxs = sortperm(feature_selection.mutual_info_classif(X, Y), rev=true)[1:50]
        for i in word_idxs
            if !(i in already_reomved_words)
                word_idx = i
                push!(already_reomved_words, i)
                break
            end
        end        
        current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
        wrd = corpus.reverse_vocabulary[word_idx]
        print("Remove word '$(wrd)' in topic $(current_topic) === ")
        TopicModels.removeWord(lda, corpus, word_idx, current_topic);
        TopicModels.gibbsSampling(lda, corpus.documents, 20);        
    end
end

Remove word 'convinces' in topic 8 === Remove word 'kingston' in topic 7 === Remove word 'batffbi' in topic 20 === Remove word 'int' in topic 10 === Remove word 'informant' in topic 4 === Remove word 'graphics' in topic 15 === Remove word 'controller' in topic 11 === Remove word 'address' in topic 13 === Remove word 'went' in topic 2 === Remove word 'ride' in topic 6 === Remove word 'streak' in topic 19 === Remove word 'flags' in topic 12 === Remove word 'medical' in topic 9 === Remove word 'legislative' in topic 16 === Remove word 'provable' in topic 18 === Remove word 'jesus' in topic 1 === Remove word 'president' in topic 14 === Remove word 'israeli' in topic 17 === Remove word 'bumped' in topic 3 === 

In [254]:
top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))

0.38732646450023844


In [255]:
#Second loop refinement for all topics
for current_cat in 1:20
    false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
    remaining = [i for i in 1:corpus.document_size if !(i in docs_ls[current_cat]) && !(i in false_docs_for_cat)]
    if false_docs_for_cat != []
        X, Y = make_PMI_dataset(false_docs_for_cat, remaining)
        word_idxs = sortperm(feature_selection.mutual_info_classif(X, Y), rev=true)[1:50]
        for i in word_idxs
            if !(i in already_reomved_words)
                word_idx = i
                push!(already_reomved_words, i)
                break
            end
        end        
        current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
        wrd = corpus.reverse_vocabulary[word_idx]
        print("Remove word '$(wrd)' in topic $(current_topic) === ")
        TopicModels.removeWord(lda, corpus, word_idx, current_topic);
        TopicModels.gibbsSampling(lda, corpus.documents, 20);        
    end
end

Remove word 'think' in topic 8 === Remove word 'mansfield' in topic 7 === Remove word 'functions' in topic 20 === Remove word 'char' in topic 10 === Remove word 'gun' in topic 4 === Remove word 'file' in topic 15 === Remove word 'ide' in topic 11 === Remove word 'arrghhh' in topic 13 === Remove word 'saw' in topic 2 === Remove word 'bike' in topic 6 === Remove word 'phillies' in topic 19 === Remove word 'hansson' in topic 12 === Remove word 'patients' in topic 9 === Remove word 'zeresan' in topic 16 === Remove word 'openwinhomesharemanusrman' in topic 18 === Remove word 'christ' in topic 1 === Remove word 'secretary' in topic 14 === Remove word 'israel' in topic 17 === Remove word 'atheism' in topic 3 === 

In [256]:
top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))

0.37546502834324186


In [257]:
#Third loop refinement for all topics
for current_cat in 1:20
    false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
    remaining = [i for i in 1:corpus.document_size if !(i in docs_ls[current_cat]) && !(i in false_docs_for_cat)]
    if false_docs_for_cat != []
        X, Y = make_PMI_dataset(false_docs_for_cat, remaining)
        word_idxs = sortperm(feature_selection.mutual_info_classif(X, Y), rev=true)[1:50]
        for i in word_idxs
            if !(i in already_reomved_words)
                word_idx = i
                push!(already_reomved_words, i)
                break
            end
        end        
        current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
        wrd = corpus.reverse_vocabulary[word_idx]
        print("Remove word '$(wrd)' in topic $(current_topic) === ")
        TopicModels.removeWord(lda, corpus, word_idx, current_topic);
        TopicModels.gibbsSampling(lda, corpus.documents, 20);        
    end
end

Remove word 'supremacists' in topic 8 === Remove word 'many' in topic 7 === Remove word 'widget' in topic 20 === Remove word 'guidance' in topic 10 === Remove word 'fbi' in topic 4 === Remove word 'ftp' in topic 15 === Remove word 'card' in topic 11 === Remove word 'steven' in topic 13 === Remove word 'said' in topic 2 === Remove word 'team' in topic 6 === Remove word 'standings' in topic 19 === Remove word 'place' in topic 12 === Remove word 'disease' in topic 9 === Remove word 'honda' in topic 16 === Remove word 'israli' in topic 18 === Remove word 'god' in topic 1 === Remove word 'stimulus' in topic 14 === Remove word 'arab' in topic 17 === Remove word 'belief' in topic 3 === 

In [258]:
top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))

0.3679746228106866


In [259]:
#Fourth loop refinement for all topics
for current_cat in 1:20
    false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
    remaining = [i for i in 1:corpus.document_size if !(i in docs_ls[current_cat]) && !(i in false_docs_for_cat)]
    if false_docs_for_cat != []
        X, Y = make_PMI_dataset(false_docs_for_cat, remaining)
        word_idxs = sortperm(feature_selection.mutual_info_classif(X, Y), rev=true)[1:50]
        for i in word_idxs
            if !(i in already_reomved_words)
                word_idx = i
                push!(already_reomved_words, i)
                break
            end
        end        
        current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
        wrd = corpus.reverse_vocabulary[word_idx]
        print("Remove word '$(wrd)' in topic $(current_topic) === ")
        TopicModels.removeWord(lda, corpus, word_idx, current_topic);
        TopicModels.gibbsSampling(lda, corpus.documents, 20);        
    end
end

Remove word 'oatesjuneau' in topic 8 === Remove word 'inefficient' in topic 7 === Remove word 'xterm' in topic 20 === Remove word 'include' in topic 10 === Remove word 'firearms' in topic 4 === Remove word 'available' in topic 15 === Remove word 'drive' in topic 11 === Remove word 'contact' in topic 13 === Remove word 'came' in topic 2 === Remove word 'sharks' in topic 6 === Remove word 'san' in topic 19 === Remove word 'contains' in topic 12 === Remove word 'diet' in topic 9 === Remove word 'relevation' in topic 16 === Remove word 'questions' in topic 18 === Remove word 'heaven' in topic 1 === Remove word 'edt' in topic 14 === Remove word 'jews' in topic 17 === Remove word 'exists' in topic 3 === 

In [260]:
top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))

0.3754477304513362


In [261]:
#Fifth loop refinement for all topics
for current_cat in 1:20
    false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
    remaining = [i for i in 1:corpus.document_size if !(i in docs_ls[current_cat]) && !(i in false_docs_for_cat)]
    if false_docs_for_cat != []
        X, Y = make_PMI_dataset(false_docs_for_cat, remaining)
        word_idxs = sortperm(feature_selection.mutual_info_classif(X, Y), rev=true)[1:50]
        for i in word_idxs
            if !(i in already_reomved_words)
                word_idx = i
                push!(already_reomved_words, i)
                break
            end
        end        
        current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
        wrd = corpus.reverse_vocabulary[word_idx]
        print("Remove word '$(wrd)' in topic $(current_topic) === ")
        TopicModels.removeWord(lda, corpus, word_idx, current_topic);
        TopicModels.gibbsSampling(lda, corpus.documents, 20);        
    end
end

Remove word 'bludgeon' in topic 8 === Remove word 'crumenam' in topic 7 === Remove word 'motif' in topic 20 === Remove word 'function' in topic 10 === Remove word 'assault' in topic 4 === Remove word 'software' in topic 15 === Remove word 'multitasking' in topic 11 === Remove word 'gumulcine' in topic 13 === Remove word 'told' in topic 2 === Remove word 'games' in topic 6 === Remove word 'sermons' in topic 19 === Remove word 'squirt' in topic 12 === Remove word 'clinical' in topic 9 === Remove word 'engine' in topic 16 === Remove word 'kind' in topic 18 === Remove word 'lord' in topic 1 === Remove word 'reform' in topic 14 === Remove word 'country' in topic 17 === Remove word 'atheist' in topic 3 === 

In [262]:
top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))

0.37433223674432287


In [264]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
already_reomved_words = [];

for loop in 1:5
    for current_cat in 1:20
        false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
        remaining = [i for i in 1:corpus.document_size if !(i in docs_ls[current_cat]) && !(i in false_docs_for_cat)]
        if false_docs_for_cat != []
            X, Y = make_PMI_dataset(false_docs_for_cat, remaining)
            word_idxs = sortperm(feature_selection.mutual_info_classif(X, Y), rev=true)[1:50]
            for i in word_idxs
                if !(i in already_reomved_words)
                    word_idx = i
                    push!(already_reomved_words, i)
                    break
                end
            end        
            current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
            wrd = corpus.reverse_vocabulary[word_idx]
            print("Remove word '$(wrd)' in topic $(current_topic) === ")
            TopicModels.removeWord(lda, corpus, word_idx, current_topic);
            TopicModels.gibbsSampling(lda, corpus.documents, 20);        
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
end

Remove word 'think' in topic 8 === Remove word 'vestments' in topic 7 === Remove word 'typex' in topic 20 === Remove word 'int' in topic 10 === Remove word 'archiveservergazoochengsuncom' in topic 4 === Remove word 'windows' in topic 15 === Remove word 'scsi' in topic 11 === Remove word 'attendees' in topic 13 === Remove word 'went' in topic 2 === Remove word 'bike' in topic 6 === Remove word 'milwaukee' in topic 19 === Remove word 'course' in topic 12 === Remove word 'patients' in topic 9 === Remove word 'yournameyourhost' in topic 16 === Remove word 'imputed' in topic 18 === Remove word 'jesus' in topic 1 === Remove word 'president' in topic 14 === Remove word 'israeli' in topic 17 === Remove word 'theists' in topic 3 === 0.3957691935979762
Remove word 'exemption' in topic 8 === Remove word 'affair' in topic 7 === Remove word 'end' in topic 20 === Remove word 'char' in topic 10 === Remove word 'police' in topic 4 === Remove word 'file' in topic 15 === Remove word 'drives' in topic 11

In [307]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
already_reomved_words = [];
t0 = time();
@time for loop in 1:5
    for current_cat in 1:20
        false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
        remaining = [i for i in 1:corpus.document_size if !(i in docs_ls[current_cat]) && !(i in false_docs_for_cat)]
        if false_docs_for_cat != []
            X, Y = make_PMI_dataset(false_docs_for_cat, remaining)
            word_idxs = sortperm(feature_selection.mutual_info_classif(X, Y), rev=true)[1:50]
            for i in word_idxs
                if !(i in already_reomved_words)
                    word_idx = i
                    push!(already_reomved_words, i)
                    break
                end
            end        
            current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
            wrd = corpus.reverse_vocabulary[word_idx]
            print("Remove word '$(wrd)' in topic $(current_topic) === ")
            TopicModels.removeWord(lda, corpus, word_idx, current_topic);
            TopicModels.gibbsSampling(lda, corpus.documents, 20);        
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
end
time() - t0

Remove word 'rubinstein' in topic 8 === Remove word 'many' in topic 7 === Remove word 'loooooooooong' in topic 20 === Remove word 'int' in topic 10 === Remove word 'lady' in topic 4 === Remove word 'file' in topic 15 === Remove word 'scsi' in topic 11 === Remove word 'contact' in topic 13 === Remove word 'went' in topic 2 === Remove word 'ride' in topic 6 === Remove word 'kansas' in topic 19 === Remove word 'several' in topic 12 === Remove word 'patients' in topic 9 === Remove word 'utilsrc' in topic 16 === Remove word 'unfavorable' in topic 18 === Remove word 'jesus' in topic 1 === Remove word 'president' in topic 14 === Remove word 'israel' in topic 17 === Remove word 'atheism' in topic 3 === 0.39458517618829964
Remove word 'think' in topic 8 === Remove word 'decompressed' in topic 7 === Remove word 'graphics' in topic 20 === Remove word 'char' in topic 10 === Remove word 'attorney' in topic 4 === Remove word 'ftp' in topic 15 === Remove word 'drive' in topic 11 === Remove word 'hote

257786.4024040699

In [308]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
already_reomved_words = [];
t0 = time();
@time for loop in 1:5
    for current_cat in 1:20
        false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
        remaining = [i for i in 1:corpus.document_size if !(i in docs_ls[current_cat]) && !(i in false_docs_for_cat)]
        if false_docs_for_cat != []
            X, Y = make_PMI_dataset(false_docs_for_cat, remaining)
            word_idxs = sortperm(feature_selection.mutual_info_classif(X, Y), rev=true)[1:50]
            for i in word_idxs
                if !(i in already_reomved_words)
                    word_idx = i
                    push!(already_reomved_words, i)
                    break
                end
            end        
            current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
            wrd = corpus.reverse_vocabulary[word_idx]
            print("Remove word '$(wrd)' in topic $(current_topic) === ")
            TopicModels.removeWord(lda, corpus, word_idx, current_topic);
            TopicModels.gibbsSampling(lda, corpus.documents, 20);        
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
end
time() - t0

Remove word 'fractures' in topic 8 === Remove word 'humphries' in topic 7 === Remove word 'noninterlaced' in topic 20 === Remove word 'int' in topic 10 === Remove word 'exaclty' in topic 4 === Remove word 'ftp' in topic 15 === Remove word 'scsi' in topic 11 === Remove word 'mail' in topic 13 === Remove word 'yelena' in topic 2 === Remove word 'ride' in topic 6 === Remove word 'standings' in topic 19 === Remove word 'one' in topic 12 === Remove word 'disease' in topic 9 === Remove word 'commemoration' in topic 16 === Remove word 'jfifformat' in topic 18 === Remove word 'jesus' in topic 1 === Remove word 'president' in topic 14 === Remove word 'israel' in topic 17 === Remove word 'atheism' in topic 3 === 0.38252950876103353
Remove word 'think' in topic 8 === Remove word 'many' in topic 7 === Remove word 'motif' in topic 20 === Remove word 'char' in topic 10 === Remove word 'compound' in topic 4 === Remove word 'graphics' in topic 15 === Remove word 'drives' in topic 11 === Remove word 'c

239452.51244187355

In [309]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
already_reomved_words = [];
t0 = time();
@time for loop in 1:5
    for current_cat in 1:20
        false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
        remaining = [i for i in 1:corpus.document_size if !(i in docs_ls[current_cat]) && !(i in false_docs_for_cat)]
        if false_docs_for_cat != []
            X, Y = make_PMI_dataset(false_docs_for_cat, remaining)
            word_idxs = sortperm(feature_selection.mutual_info_classif(X, Y), rev=true)[1:50]
            for i in word_idxs
                if !(i in already_reomved_words)
                    word_idx = i
                    push!(already_reomved_words, i)
                    break
                end
            end        
            current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
            wrd = corpus.reverse_vocabulary[word_idx]
            print("Remove word '$(wrd)' in topic $(current_topic) === ")
            TopicModels.removeWord(lda, corpus, word_idx, current_topic);
            TopicModels.gibbsSampling(lda, corpus.documents, 20);        
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
end
time() - t0

Remove word 'kraatveldstraat' in topic 8 === Remove word 'punditseditors' in topic 7 === Remove word 'headtohead' in topic 20 === Remove word 'int' in topic 10 === Remove word 'noonan' in topic 4 === Remove word 'file' in topic 15 === Remove word 'scsi' in topic 11 === Remove word 'mail' in topic 13 === Remove word 'went' in topic 2 === Remove word 'bike' in topic 6 === Remove word 'cubs' in topic 19 === Remove word 'disease' in topic 9 === Remove word 'tensioners' in topic 16 === Remove word 'rabbi' in topic 18 === Remove word 'jesus' in topic 1 === Remove word 'president' in topic 14 === Remove word 'israeli' in topic 17 === Remove word 'luggers' in topic 3 === 0.39498399806502404
Remove word 'think' in topic 8 === Remove word 'enligthen' in topic 7 === Remove word 'programming' in topic 20 === Remove word 'null' in topic 10 === Remove word 'establish' in topic 4 === Remove word 'software' in topic 15 === Remove word 'ide' in topic 11 === Remove word 'ricks' in topic 13 === Remove wo

239235.47860503197

In [310]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
already_reomved_words = [];
t0 = time();
@time for loop in 1:5
    for current_cat in 1:20
        false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
        remaining = [i for i in 1:corpus.document_size if !(i in docs_ls[current_cat]) && !(i in false_docs_for_cat)]
        if false_docs_for_cat != []
            X, Y = make_PMI_dataset(false_docs_for_cat, remaining)
            word_idxs = sortperm(feature_selection.mutual_info_classif(X, Y), rev=true)[1:50]
            for i in word_idxs
                if !(i in already_reomved_words)
                    word_idx = i
                    push!(already_reomved_words, i)
                    break
                end
            end        
            current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
            wrd = corpus.reverse_vocabulary[word_idx]
            print("Remove word '$(wrd)' in topic $(current_topic) === ")
            TopicModels.removeWord(lda, corpus, word_idx, current_topic);
            TopicModels.gibbsSampling(lda, corpus.documents, 20);        
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
end
time() - t0

Remove word 'think' in topic 8 === Remove word 'juggle' in topic 7 === Remove word 'farah' in topic 20 === Remove word 'int' in topic 10 === Remove word 'informant' in topic 4 === Remove word 'windows' in topic 15 === Remove word 'drive' in topic 11 === Remove word 'contact' in topic 13 === Remove word 'went' in topic 2 === Remove word 'ride' in topic 6 === Remove word 'cubs' in topic 19 === Remove word 'end' in topic 12 === Remove word 'patients' in topic 9 === Remove word 'decongestant' in topic 16 === Remove word 'unresurrected' in topic 18 === Remove word 'jesus' in topic 1 === Remove word 'president' in topic 14 === Remove word 'israel' in topic 17 === Remove word 'religious' in topic 3 === 0.39719774715467226
Remove word 'commissioner' in topic 8 === Remove word 'many' in topic 7 === Remove word 'pixel' in topic 20 === Remove word 'char' in topic 10 === Remove word 'fbi' in topic 4 === Remove word 'file' in topic 15 === Remove word 'scsi' in topic 11 === Remove word 'address' in 

204185.39231991768

In [12]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
already_reomved_words = [];
t0 = time();
word_idx = -1
@time for loop in 1:5
    for current_cat in 1:20
        false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
        remaining = [i for i in 1:corpus.document_size if !(i in docs_ls[current_cat]) && !(i in false_docs_for_cat)]
        if false_docs_for_cat != []
            X, Y = make_PMI_dataset(false_docs_for_cat, remaining)
            word_idxs = sortperm(feature_selection.mutual_info_classif(X, Y), rev=true)[1:50]
            for i in word_idxs
                if !(i in already_reomved_words)
                    word_idx = i
                    push!(already_reomved_words, i)
                    break
                end
            end        
            current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
            wrd = corpus.reverse_vocabulary[word_idx]
            print("Remove word '$(wrd)' in topic $(current_topic) === ")
            TopicModels.removeWord(lda, corpus, word_idx, current_topic);
            TopicModels.gibbsSampling(lda, corpus.documents, 20);        
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
end
time() - t0

Remove word 'think' in topic 8 === Remove word 'sharon' in topic 7 === Remove word 'xmnwidth' in topic 20 === Remove word 'int' in topic 10 === Remove word 'prison' in topic 4 === Remove word 'software' in topic 15 === Remove word 'ide' in topic 11 === Remove word 'information' in topic 13 === Remove word 'went' in topic 2 === Remove word 'bike' in topic 6 === Remove word 'cincinnati' in topic 19 === Remove word 'things' in topic 12 === Remove word 'patients' in topic 9 === Remove word 'coombes' in topic 16 === Remove word 'vettesrequestcompaqcom' in topic 18 === Remove word 'jesus' in topic 1 === Remove word 'president' in topic 14 === Remove word 'israeli' in topic 17 === Remove word 'theism' in topic 3 === 0.3918033516862313
Remove word 'sanction' in topic 8 === Remove word 'many' in topic 7 === Remove word 'widget' in topic 20 === Remove word 'arg' in topic 10 === Remove word 'gun' in topic 4 === Remove word 'workstation' in topic 15 === Remove word 'scsi' in topic 11 === Remove wo

56799.0968477726

In [13]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
already_reomved_words = [];
t0 = time();
word_idx = -1
@time for loop in 1:5
    for current_cat in 1:20
        false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
        remaining = [i for i in 1:corpus.document_size if !(i in docs_ls[current_cat]) && !(i in false_docs_for_cat)]
        if false_docs_for_cat != []
            X, Y = make_PMI_dataset(false_docs_for_cat, remaining)
            word_idxs = sortperm(feature_selection.mutual_info_classif(X, Y), rev=true)[1:50]
            for i in word_idxs
                if !(i in already_reomved_words)
                    word_idx = i
                    push!(already_reomved_words, i)
                    break
                end
            end        
            current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
            wrd = corpus.reverse_vocabulary[word_idx]
            print("Remove word '$(wrd)' in topic $(current_topic) === ")
            TopicModels.removeWord(lda, corpus, word_idx, current_topic);
            TopicModels.gibbsSampling(lda, corpus.documents, 20);        
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
end
time() - t0

Remove word 'think' in topic 8 === Remove word 'upsurge' in topic 7 === Remove word 'offices' in topic 20 === Remove word 'int' in topic 10 === Remove word 'koblitz' in topic 4 === Remove word 'graphics' in topic 15 === Remove word 'scsi' in topic 11 === Remove word 'mail' in topic 13 === Remove word 'saw' in topic 2 === Remove word 'bike' in topic 6 === Remove word 'oakland' in topic 19 === Remove word 'living' in topic 12 === Remove word 'patients' in topic 9 === Remove word 'mamograma' in topic 16 === Remove word 'alarms' in topic 18 === Remove word 'jesus' in topic 1 === Remove word 'president' in topic 14 === Remove word 'israel' in topic 17 === Remove word 'religious' in topic 3 === 0.40022796010062034
Remove word 'crumbles' in topic 8 === Remove word 'saturnian' in topic 7 === Remove word 'triangles' in topic 20 === Remove word 'char' in topic 10 === Remove word 'investigator' in topic 4 === Remove word 'sites' in topic 15 === Remove word 'card' in topic 11 === Remove word 'pict

57831.24010515213

In [14]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
already_reomved_words = [];
t0 = time();
word_idx = -1
@time for loop in 1:5
    for current_cat in 1:20
        false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
        remaining = [i for i in 1:corpus.document_size if !(i in docs_ls[current_cat]) && !(i in false_docs_for_cat)]
        if false_docs_for_cat != []
            X, Y = make_PMI_dataset(false_docs_for_cat, remaining)
            word_idxs = sortperm(feature_selection.mutual_info_classif(X, Y), rev=true)[1:50]
            for i in word_idxs
                if !(i in already_reomved_words)
                    word_idx = i
                    push!(already_reomved_words, i)
                    break
                end
            end        
            current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
            wrd = corpus.reverse_vocabulary[word_idx]
            print("Remove word '$(wrd)' in topic $(current_topic) === ")
            TopicModels.removeWord(lda, corpus, word_idx, current_topic);
            TopicModels.gibbsSampling(lda, corpus.documents, 20);        
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
end
time() - t0

Remove word 'reinnoculation' in topic 8 === Remove word 'stretch' in topic 7 === Remove word 'kidnapping' in topic 20 === Remove word 'int' in topic 10 === Remove word 'prison' in topic 4 === Remove word 'software' in topic 15 === Remove word 'scsi' in topic 11 === Remove word 'xtappprocessevent' in topic 13 === Remove word 'went' in topic 2 === Remove word 'bike' in topic 6 === Remove word 'montreal' in topic 19 === Remove word 'treat' in topic 12 === Remove word 'patients' in topic 9 === Remove word 'berthiaume' in topic 16 === Remove word 'valueless' in topic 18 === Remove word 'jesus' in topic 1 === Remove word 'president' in topic 14 === Remove word 'israel' in topic 17 === Remove word 'atheism' in topic 3 === 0.389791414139785
Remove word 'think' in topic 8 === Remove word 'remind' in topic 7 === Remove word 'widgets' in topic 20 === Remove word 'char' in topic 10 === Remove word 'fbi' in topic 4 === Remove word 'graphics' in topic 15 === Remove word 'controller' in topic 11 === 

60226.438605070114

In [15]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
already_reomved_words = [];
t0 = time();
word_idx = -1
@time for loop in 1:5
    for current_cat in 1:20
        false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
        remaining = [i for i in 1:corpus.document_size if !(i in docs_ls[current_cat]) && !(i in false_docs_for_cat)]
        if false_docs_for_cat != []
            X, Y = make_PMI_dataset(false_docs_for_cat, remaining)
            word_idxs = sortperm(feature_selection.mutual_info_classif(X, Y), rev=true)[1:50]
            for i in word_idxs
                if !(i in already_reomved_words)
                    word_idx = i
                    push!(already_reomved_words, i)
                    break
                end
            end        
            current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
            wrd = corpus.reverse_vocabulary[word_idx]
            print("Remove word '$(wrd)' in topic $(current_topic) === ")
            TopicModels.removeWord(lda, corpus, word_idx, current_topic);
            TopicModels.gibbsSampling(lda, corpus.documents, 20);        
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
end
time() - t0

Remove word 'think' in topic 8 === Remove word 'many' in topic 7 === Remove word 'irulesrcsitedef' in topic 20 === Remove word 'int' in topic 10 === Remove word 'told' in topic 4 === Remove word 'windows' in topic 15 === Remove word 'drive' in topic 11 === Remove word 'please' in topic 13 === Remove word 'went' in topic 2 === Remove word 'bike' in topic 6 === Remove word 'seattle' in topic 19 === Remove word 'everything' in topic 12 === Remove word 'medical' in topic 9 === Remove word 'raises' in topic 16 === Remove word 'instructs' in topic 18 === Remove word 'jesus' in topic 1 === Remove word 'president' in topic 14 === Remove word 'israeli' in topic 17 === Remove word 'programmmers' in topic 3 === 0.3927015509705446
Remove word 'sorehandvmucsfedu' in topic 8 === Remove word 'pouches' in topic 7 === Remove word 'polyline' in topic 20 === Remove word 'char' in topic 10 === Remove word 'fbi' in topic 4 === Remove word 'file' in topic 15 === Remove word 'ide' in topic 11 === Remove word

56739.99833512306

In [4]:
function make_PMI_dataset(this_y_1, this_y_0)
    X = []; Y = []
    for i in this_y_1
        push!(X, position_to_feature(i))
        push!(Y, 1)
    end
    for i in this_y_0
        push!(X, position_to_feature(i))
        push!(Y, 0)
    end
    return X, Y
end

make_PMI_dataset (generic function with 1 method)

In [5]:
function position_to_feature(doc_idx)
    position_vector = corpus.documents[doc_idx]
    feature_vector = [0 for i in 1:corpus.vocab_count]
    for (idx, val) in enumerate(position_vector)
        feature_vector[val] +=1
    end
    return feature_vector
end

position_to_feature (generic function with 1 method)

In [6]:
function false_docs_in_cat(lda, corpus, docs_ls, cat_idx)
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    topic_x_top_docs = [idx for (idx, val) in enumerate(predicted_with_marriage_assigment) if val==cat_idx];
    docs_in_topic_notin_cat = []
    for i in topic_x_top_docs 
        if !(i in docs_ls[cat_idx])
            push!(docs_in_topic_notin_cat, i)
        end
    end
    return docs_in_topic_notin_cat
end

false_docs_in_cat (generic function with 1 method)

In [7]:
function topic_of_each_doc(lda, corpus)
    top_topic_for_each_doc = []
    for i in 1:corpus.document_size
        top_val = 0
        top_topic = 0
        for j in 1:lda.M
            v = TopicModels.topicPredict(lda, i, j)
            if v>=top_val
                top_val = v
                top_topic = j
            end
        end
        push!(top_topic_for_each_doc, top_topic)
    end
    return top_topic_for_each_doc
end

topic_of_each_doc (generic function with 1 method)

## Add_word refinement after stable marriage algorithm

In [166]:
true_cat_original = [17, 7, 18, 17, 10, 19, 16, 19, 18, 8, 13, 7, 12, 9, 18, 18, 17, 17, 15, 17, 7, 1, 19, 5, 4, 8, 19, 17, 15, 15, 17, 0, 19, 11, 17, 18, 9, 13, 19, 17, 13, 17, 19, 15, 7, 8, 11, 9, 1, 8, 0, 18, 17, 14, 14, 10, 15, 3, 8, 10, 10, 11, 17, 6, 17, 17, 15, 8, 15, 11, 2, 17, 9, 18, 17, 17, 10, 2, 9, 1, 10, 4, 9, 15, 3, 15, 19, 15, 16, 13, 17, 11, 0, 18, 10, 14, 6, 17, 15, 17, 19, 14, 8, 14, 5, 5, 11, 15, 6, 10, 5, 9, 15, 11, 9, 15, 7, 1, 14, 16, 15, 13, 13, 17, 0, 11, 16, 3, 16, 15, 14, 14, 19, 9, 18, 14, 17, 17, 4, 17, 15, 1, 16, 11, 15, 6, 2, 14, 15, 18, 8, 17, 18, 4, 7, 11, 17, 18, 9, 16, 18, 7, 11, 2, 15, 19, 1, 11, 14, 17, 1, 16, 6, 5, 19, 7, 11, 16, 8, 16, 10, 15, 17, 16, 16, 1, 0, 7, 15, 0, 6, 18, 10, 9, 11, 16, 19, 14, 6, 0, 0, 3, 17, 17, 12, 16, 8, 6, 19, 10, 17, 16, 14, 4, 16, 11, 1, 14, 0, 2, 16, 15, 15, 6, 15, 0, 14, 10, 15, 13, 2, 19, 15, 17, 15, 1, 9, 16, 18, 17, 6, 17, 8, 14, 11, 5, 15, 7, 15, 15, 12, 11, 14, 14, 3, 15, 3, 17, 3, 14, 11, 19, 10, 18, 15, 16, 15, 15, 9, 16, 11, 4, 19, 13, 3, 19, 9, 17, 18, 7, 7, 15, 16, 14, 15, 10, 5, 9, 14, 14, 17, 15, 15, 19, 14, 15, 15, 0, 4, 11, 11, 15, 12, 6, 3, 15, 14, 5, 14, 18, 11, 11, 14, 17, 17, 10, 9, 5, 7, 10, 19, 18, 17, 15, 18, 17, 11, 11, 13, 3, 2, 15, 10, 17, 2, 14, 13, 15, 15, 16, 15, 13, 14, 7, 16, 19, 3, 17, 17, 17, 14, 16, 9, 8, 14, 14, 16, 13, 12, 11, 17, 11, 1, 4, 11, 18, 16, 15, 11, 17, 15, 6, 17, 17, 13, 9, 10, 18, 17, 17, 13, 6, 1, 9, 14, 19, 3, 19, 13, 14, 10, 16, 19, 14, 9, 17, 7, 7, 11, 17, 18, 11, 17, 11, 18, 17, 16, 14, 1, 11, 16, 11, 18, 6, 1, 14, 15, 6, 17, 18, 16, 6, 13, 11, 0, 0, 16, 8, 17, 17, 10, 4, 15, 15, 15, 1, 13, 15, 14, 5, 15, 9, 1, 0, 13, 11, 1, 14, 16, 7, 2, 6, 10, 8, 14, 12, 19, 2, 7, 15, 17, 13, 17, 13, 17, 17, 19, 16, 15, 0, 12, 19, 19, 18, 15, 16, 9, 15, 16, 16, 12, 9, 9, 18, 4, 18, 11, 17, 15, 12, 12, 17, 17, 15, 19, 15, 19, 17, 15, 10, 14, 19, 16, 17, 10, 10, 14, 16, 5, 13, 9, 15, 17, 15, 19, 0, 18, 14, 15, 1, 15, 5, 17, 12, 18, 10, 5, 16, 0, 15, 1, 5, 7, 11, 19, 13, 11, 17, 9, 18, 9, 4, 10, 11, 5, 15, 12, 15, 1, 18, 6, 13, 18, 9, 1, 17, 18, 17, 15, 15, 19, 3, 17, 15, 9, 11, 14, 17, 14, 15, 1, 15, 15, 10, 3, 19, 17, 14, 6, 11, 12, 15, 17, 15, 13, 3, 0, 10, 4, 11, 0, 14, 7, 0, 15, 0, 2, 17, 17, 6, 19, 14, 17, 14, 6, 15, 12, 13, 4, 1, 17, 16, 17, 13, 18, 0, 11, 10, 3, 0, 13, 15, 13, 15, 15, 13, 5, 18, 1, 11, 19, 15, 13, 13, 10, 17, 13, 14, 19, 14, 0, 5, 7, 15, 11, 1, 15, 15, 14, 15, 13, 0, 15, 18, 7, 8, 13, 11, 16, 14, 19, 15, 16, 0, 4, 17, 11, 10, 11, 5, 19, 10, 17, 7, 10, 17, 9, 11, 15, 19, 17, 11, 11, 11, 10, 16, 14, 17, 18, 13, 16, 19, 9, 11, 18, 16, 1, 13, 16, 18, 16, 11, 16, 8, 14, 14, 16, 15, 13, 15, 11, 11, 16, 2, 15, 16, 19, 8, 17, 19, 0, 16, 15, 14, 17, 9, 19, 6, 10, 16, 15, 0, 17, 10, 6, 16, 16, 17, 11, 15, 14, 10, 2, 18, 0, 18, 16, 10, 8, 3, 17, 17, 15, 11, 17, 13, 9, 18, 19, 11, 17, 11, 11, 5, 18, 17, 13, 11, 14, 13, 17, 15, 2, 0, 12, 0, 0, 14, 15, 8, 18, 9, 17, 10, 15, 13, 13, 3, 4, 16, 10, 5, 2, 2, 16, 17, 15, 7, 15, 17, 16, 12, 17, 15, 16, 15, 17, 17, 17, 18, 19, 17, 14, 18, 11, 12, 18, 6, 14, 11, 0, 15, 10, 17, 3, 11, 9, 15, 11, 9, 13, 16, 15, 13, 17, 18, 10, 4, 18, 19, 5, 19, 17, 11, 10, 19, 17, 19, 18, 10, 3, 13, 10, 16, 11, 5, 0, 3, 11, 17, 13, 14, 6, 16, 4, 19, 19, 14, 1, 14, 16, 11, 6, 18, 19, 11, 10, 11, 19, 11, 9, 19, 10, 17, 10, 18, 14, 11, 9, 0, 9, 1, 15, 11, 8, 14, 18, 11, 16, 15, 16, 6, 18, 13, 18, 2, 17, 15, 16, 15, 11, 16, 5, 15, 11, 14, 1, 16, 15, 11, 19, 16, 11, 15, 17, 15, 2, 16, 11, 7, 13, 17, 16, 7, 10, 1, 16, 1, 18, 17, 11, 17, 11, 5, 17, 1, 18, 0, 18, 16, 17, 12, 17, 11, 11, 15, 15, 12, 0, 19, 10, 17, 14, 12, 0, 19, 11, 2, 16, 19, 17, 15, 10, 5, 16, 5, 5, 13, 11, 19, 4, 15, 14, 11, 17, 15, 14, 8, 17, 3, 13, 0, 17, 0, 10, 12, 14, 6, 13, 13, 11, 10, 13, 10, 14, 0, 19, 11, 7, 10, 14, 3, 15, 3, 14, 10, 17, 4, 5, 11, 0, 19, 10, 10, 4, 1, 13, 17, 7, 5, 19, 13, 11, 14, 19, 19, 18, 2, 13, 6, 9, 0, 11, 11, 6, 11, 18, 16, 14, 10, 2, 8, 16, 5, 17, 16, 13, 18, 17, 13, 18, 15, 19, 6, 18, 17, 19, 12, 18, 9, 13, 6, 0, 15, 11, 18, 11, 0, 18, 15, 10, 10, 6, 5, 17, 11, 1, 9, 1, 14, 12, 14, 10, 15, 11, 11, 18, 2, 10, 3, 17, 18, 14, 19, 17, 15, 9, 5, 1, 9, 13, 15, 13, 10, 17, 19, 4, 17, 17, 15, 10, 17, 13, 18, 17, 11, 18, 18, 15, 15, 17, 11, 10, 8, 0, 15, 15, 14, 11, 15, 11, 8, 15, 3, 12, 8, 15, 14, 15, 10, 10, 11, 11, 3, 15, 9, 19, 17, 4, 7, 15, 4, 9, 9, 1, 6, 13, 10, 19, 9, 11, 16, 19, 17, 16, 14, 10, 9, 15, 15, 9, 5, 0, 2, 18, 9, 15, 19, 19, 17, 5, 10, 14, 15, 17, 15, 18, 17, 15, 4, 8, 13, 8, 11, 1, 15, 13, 15, 9, 9, 17, 14, 10, 14, 0, 15, 0, 17, 10, 0, 17, 0, 10, 14, 0, 12, 16, 17, 13, 13, 19, 3, 15, 0, 16, 19, 11, 14, 17, 19, 0, 14, 2, 18, 17, 15, 2, 17, 3, 11, 14, 9, 4, 3, 9, 0, 11, 13, 5, 15, 10, 14, 10, 15, 0, 9, 17, 13, 0, 5, 17, 16, 7, 0, 19, 15, 18, 15, 17, 0, 1, 15, 11, 17, 15, 9, 18, 6, 11, 17, 12, 13, 15, 18, 7, 17, 15, 15, 6, 13, 11, 18, 4, 5, 13, 7, 17, 18, 17, 15, 18, 0, 6, 5, 18, 13, 14, 2, 19, 8, 8, 10, 8, 17, 11, 15, 10, 17, 0, 19, 8, 6, 15, 12, 15, 16, 12, 19, 6, 6, 9, 14, 14, 17, 16, 16, 13, 6, 13, 15, 6, 15, 0, 10, 10, 15, 13, 14, 15, 17, 10, 15, 17, 11, 5, 9, 12, 16, 10, 13, 16, 2, 17, 6, 5, 0, 15, 9, 15, 19, 0, 11, 18, 1, 3, 18, 18, 18, 17, 0, 4, 17, 15, 15, 17, 1, 1, 15, 1, 15, 15, 16, 19, 17, 15, 2, 17, 15, 11, 16, 11, 15, 12, 15, 6, 13, 16, 19, 3, 5, 5, 7, 6, 1, 1, 16, 15, 18, 0, 15, 0, 15, 17, 19, 15, 0, 16, 14, 17, 6, 7, 15, 11, 12, 15, 15, 17, 15, 12, 17, 1, 1, 14, 3, 6, 17, 17, 18, 15, 1, 6, 10, 18, 10, 2, 11, 13, 17, 16, 3, 17, 19, 19, 18, 15, 8, 5, 16, 11, 0, 9, 13, 0, 9, 10, 1, 15, 1, 10, 17, 1, 14, 16, 1, 14, 15, 11, 14, 6, 8, 15, 9, 19, 16, 7, 17, 14, 17, 15, 18, 5, 11, 15, 2, 16, 5, 15, 9, 12, 17, 10, 15, 17, 19, 9, 1, 15, 17, 18, 3, 16, 18, 1, 13, 3, 18, 18, 15, 17, 17, 9, 19, 3, 11, 16, 17, 10, 1, 7, 19, 9, 19, 1, 17, 10, 18, 17, 8, 10, 17, 19, 13, 5, 18, 15, 11, 19, 0, 14, 11, 15, 15, 18, 18, 16, 13, 2, 16, 12, 5, 6, 0, 5, 15, 3, 16, 8, 8, 1, 17, 13, 18, 17, 17, 14, 6, 16, 14, 4, 3, 3, 3, 17, 7, 4, 14, 15, 18, 11, 16, 17, 15, 17, 6, 9, 13, 17, 6, 17, 18, 3, 17, 15, 3, 15, 18, 10, 17, 2, 18, 11, 13, 11, 15, 1, 10, 18, 19, 14, 11, 18, 15, 0, 1, 15, 17, 15, 8, 11, 18, 1, 7, 19, 10, 0, 9, 17, 9, 17, 11, 1, 19, 7, 15, 13, 17, 17, 0, 9, 16, 18, 18, 13, 15, 5, 7, 15, 15, 4, 15, 5, 5, 8, 9, 11, 13, 12, 16, 17, 17, 1, 19, 15, 14, 13, 11, 13, 13, 13, 14, 2, 9, 15, 7, 15, 2, 8, 11, 15, 11, 0, 4, 11, 15, 0, 15, 1, 5, 17, 5, 11, 5, 4, 8, 17, 3, 5, 14, 5, 13, 10, 16, 17, 9, 7, 6, 10, 10, 19, 3, 14, 17, 13, 1, 16, 0, 7, 17, 15, 0, 11, 15, 19, 16, 18, 0, 10, 17, 1, 17, 1, 15, 15, 9, 12, 14, 15, 11, 10, 11, 16, 13, 17, 15, 2, 13, 19, 19, 15, 16, 10, 19, 6, 14, 17, 13, 19, 15, 6, 17, 16, 15, 17, 18, 7, 12, 14, 17, 13, 1, 5, 9, 0, 6, 4, 17, 4, 13, 17, 16, 19, 1, 6, 14, 11, 11, 9, 14, 10, 16, 3, 12, 17, 15, 17, 1, 3, 13, 14, 10, 5, 14, 15, 6, 10, 15, 16, 14, 15, 15, 9, 1, 17, 15, 19, 17, 2, 9, 9, 1, 15, 17, 17, 15, 16, 16, 14, 9, 17, 17, 10, 4, 15, 17, 16, 18, 18, 19, 17, 3, 12, 7, 11, 3, 13, 12, 15, 1, 19, 17, 12, 16, 9, 8, 14, 0, 16, 5, 10, 15, 15, 15, 12, 17, 18, 15, 14, 2, 15, 18, 0, 15, 4, 1, 1, 1, 13, 13, 3, 16, 9, 19, 17, 12, 10, 18, 0, 0, 14, 7, 13, 14, 17, 13, 17, 1, 6, 9, 15, 4, 15, 8, 17, 18, 7, 19, 16, 9, 6, 16, 10, 14, 6, 18, 18, 11, 17, 7, 18, 13, 3, 17, 17, 15, 19, 19, 11, 0, 17, 15, 19, 18, 15, 5, 10, 0, 0, 0, 1, 13, 0, 12, 18, 8, 0, 2, 15, 17, 18, 6, 17, 13, 15, 0, 17, 18, 15, 18, 13, 19, 10, 8, 15, 2, 1, 15, 17, 15, 17, 17]
docs_cat = [i+1 for i in true_cat_original];
docs_ls = [[] for i in 1:20]
for (idx, val) in enumerate(docs_cat)
    push!(docs_ls[val], idx)
end

In [167]:
topic_marriage_assigment = Dict(1=>16, 2=>9, 3=>20, 4=>5, 5=>18, 6=>10, 7=>2, 8=>1, 9=>13, 10=>4, 
                            11=>7, 12=>12, 13=>8, 14=>17, 15=>6, 16=>14, 17=>19, 18=>15, 19=>11, 20=>3);

In [214]:
already_done_words = []

Any[]

In [216]:
# First loop of add_word refinement
for current_cat in 1:20
    true_docs, absent_docs = docs_in_and_notin_cat(lda, corpus, current_cat)
    all_false_negatives = [i for i in 1:corpus.document_size if !(i in docs_ls[current_cat])]
    if absent_docs != []
        X, Y = make_PMI_dataset(absent_docs, all_false_negatives)
        word_idxs = sortperm(feature_selection.mutual_info_classif(X, Y), rev=true)[1:50]
        for i in word_idxs
            if !(i in already_done_words)
                word_idx = i
                push!(already_done_words, i)
                break
            end
        end
        current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
        wrd = corpus.reverse_vocabulary[word_idx]
        print("Add word '$(wrd)' in topic $(current_topic) === ")
        TopicModels.addWord(lda, corpus, word_idx, current_topic)
        TopicModels.gibbsSampling(lda, corpus.documents, 20);
    end
end

Add this word 'perpetrators' in topic 8 === Add this word 'graphics' in topic 7 === Add this word 'windows' in topic 20 === Add this word 'drive' in topic 10 === Add this word 'apple' in topic 4 === Add this word 'int' in topic 15 === Add this word 'shipping' in topic 11 === Add this word 'car' in topic 13 === Add this word 'bike' in topic 2 === Add this word 'braves' in topic 6 === Add this word 'team' in topic 19 === Add this word 'encryption' in topic 12 === Add this word 'asente' in topic 9 === Add this word 'medical' in topic 16 === Add this word 'space' in topic 18 === Add this word 'god' in topic 1 === Add this word 'gun' in topic 14 === Add this word 'israeli' in topic 5 === Add this word 'president' in topic 17 === Add this word 'jesus' in topic 3 === 

In [217]:
top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))

0.403893761674023


In [219]:
# Second loop of add_word refinement
for current_cat in 1:20
    true_docs, absent_docs = docs_in_and_notin_cat(lda, corpus, current_cat)
    all_false_negatives = [i for i in 1:corpus.document_size if !(i in docs_ls[current_cat])]
    if absent_docs != []
        X, Y = make_PMI_dataset(absent_docs, all_false_negatives)
        word_idxs = sortperm(feature_selection.mutual_info_classif(X, Y), rev=true)[1:50]
        for i in word_idxs
            if !(i in already_done_words)
                word_idx = i
                push!(already_done_words, i)
                break
            end
        end
        current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
        wrd = corpus.reverse_vocabulary[word_idx]
        print("Add word '$(wrd)' in topic $(current_topic) === ")
        TopicModels.addWord(lda, corpus, word_idx, current_topic)
        TopicModels.gibbsSampling(lda, corpus.documents, 20);
    end
end

Add word 'atheism' in topic 8 === Add word 'computer' in topic 7 === Add word 'dos' in topic 20 === Add word 'drives' in topic 10 === Add word 'mac' in topic 4 === Add word 'char' in topic 15 === Add word 'sale' in topic 11 === Add word 'cars' in topic 13 === Add word 'bikes' in topic 2 === Add word 'jays' in topic 6 === Add word 'hockey' in topic 19 === Add word 'clipper' in topic 12 === Add word 'atarist' in topic 9 === Add word 'diet' in topic 16 === Add word 'nasa' in topic 18 === Add word 'bible' in topic 1 === Add word 'guns' in topic 14 === Add word 'israel' in topic 5 === Add word 'wannabe' in topic 17 === Add word 'bitmaps' in topic 3 === 

In [220]:
top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))

0.4056407508502718


In [221]:
# Third loop of add_word refinement
for current_cat in 1:20
    true_docs, absent_docs = docs_in_and_notin_cat(lda, corpus, current_cat)
    all_false_negatives = [i for i in 1:corpus.document_size if !(i in docs_ls[current_cat])]
    if absent_docs != []
        X, Y = make_PMI_dataset(absent_docs, all_false_negatives)
        word_idxs = sortperm(feature_selection.mutual_info_classif(X, Y), rev=true)[1:50]
        for i in word_idxs
            if !(i in already_done_words)
                word_idx = i
                push!(already_done_words, i)
                break
            end
        end
        current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
        wrd = corpus.reverse_vocabulary[word_idx]
        print("Add word '$(wrd)' in topic $(current_topic) === ")
        TopicModels.addWord(lda, corpus, word_idx, current_topic)
        TopicModels.gibbsSampling(lda, corpus.documents, 20);
    end
end

Add word 'atheist' in topic 8 === Add word 'image' in topic 7 === Add word 'applications' in topic 20 === Add word 'ide' in topic 10 === Add word 'iisi' in topic 4 === Add word 'function' in topic 15 === Add word 'secretaries' in topic 11 === Add word 'engine' in topic 13 === Add word 'motorcycle' in topic 2 === Add word 'marlins' in topic 6 === Add word 'nhl' in topic 19 === Add word 'secure' in topic 12 === Add word 'gslv' in topic 9 === Add word 'patients' in topic 16 === Add word 'sky' in topic 18 === Add word 'christianity' in topic 1 === Add word 'firearms' in topic 14 === Add word 'arab' in topic 5 === Add word 'care' in topic 17 === Add word 'brings' in topic 3 === 

In [222]:
top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))

0.4153138619598633


In [223]:
# Fourth loop of add_word refinement
for current_cat in 1:20
    true_docs, absent_docs = docs_in_and_notin_cat(lda, corpus, current_cat)
    all_false_negatives = [i for i in 1:corpus.document_size if !(i in docs_ls[current_cat])]
    if absent_docs != []
        X, Y = make_PMI_dataset(absent_docs, all_false_negatives)
        word_idxs = sortperm(feature_selection.mutual_info_classif(X, Y), rev=true)[1:50]
        for i in word_idxs
            if !(i in already_done_words)
                word_idx = i
                push!(already_done_words, i)
                break
            end
        end
        current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
        wrd = corpus.reverse_vocabulary[word_idx]
        print("Add word '$(wrd)' in topic $(current_topic) === ")
        TopicModels.addWord(lda, corpus, word_idx, current_topic)
        TopicModels.gibbsSampling(lda, corpus.documents, 20);
    end
end

Add word 'belief' in topic 8 === Add word 'manipulation' in topic 7 === Add word 'file' in topic 20 === Add word 'controller' in topic 10 === Add word 'macs' in topic 4 === Add word 'binsh' in topic 15 === Add word 'flmurdps' in topic 11 === Add word 'ford' in topic 13 === Add word 'riding' in topic 2 === Add word 'phillies' in topic 6 === Add word 'playoffs' in topic 19 === Add word 'crypto' in topic 12 === Add word 'kinetic' in topic 9 === Add word 'medicine' in topic 16 === Add word 'shuttle' in topic 18 === Add word 'christians' in topic 1 === Add word 'weapons' in topic 14 === Add word 'palestinians' in topic 5 === Add word 'economy' in topic 17 === Add word 'christ' in topic 3 === 

In [224]:
top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))

0.41403009246617767


In [225]:
# Fifth loop of add_word refinement
for current_cat in 1:20
    true_docs, absent_docs = docs_in_and_notin_cat(lda, corpus, current_cat)
    all_false_negatives = [i for i in 1:corpus.document_size if !(i in docs_ls[current_cat])]
    if absent_docs != []
        X, Y = make_PMI_dataset(absent_docs, all_false_negatives)
        word_idxs = sortperm(feature_selection.mutual_info_classif(X, Y), rev=true)[1:50]
        for i in word_idxs
            if !(i in already_done_words)
                word_idx = i
                push!(already_done_words, i)
                break
            end
        end
        current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
        wrd = corpus.reverse_vocabulary[word_idx]
        print("Add word '$(wrd)' in topic $(current_topic) === ")
        TopicModels.addWord(lda, corpus, word_idx, current_topic)
        TopicModels.gibbsSampling(lda, corpus.documents, 20);
    end
end

Add word 'atheists' in topic 8 === Add word 'processing' in topic 7 === Add word 'compmailmh' in topic 20 === Add word 'card' in topic 10 === Add word 'shading' in topic 4 === Add word 'ped' in topic 15 === Add word 'cod' in topic 11 === Add word 'glass' in topic 13 === Add word 'ride' in topic 2 === Add word 'royals' in topic 6 === Add word 'drugresistant' in topic 19 === Add word 'privacy' in topic 12 === Add word 'snf' in topic 9 === Add word 'disease' in topic 16 === Add word 'orbit' in topic 18 === Add word 'christian' in topic 1 === Add word 'handgun' in topic 14 === Add word 'jews' in topic 5 === Add word 'taxes' in topic 17 === Add word 'abiding' in topic 3 === 

In [226]:
top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))

0.3940091516828013


In [266]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
already_done_words = [];

for loop in 1:5
    for current_cat in 1:20
        true_docs, absent_docs = docs_in_and_notin_cat(lda, corpus, current_cat)
        all_false_negatives = [i for i in 1:corpus.document_size if !(i in docs_ls[current_cat])]
        if absent_docs != []
            X, Y = make_PMI_dataset(absent_docs, all_false_negatives)
            word_idxs = sortperm(feature_selection.mutual_info_classif(X, Y), rev=true)[1:50]
            for i in word_idxs
                if !(i in already_done_words)
                    word_idx = i
                    push!(already_done_words, i)
                    break
                end
            end
            current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
            wrd = corpus.reverse_vocabulary[word_idx]
            print("Add word '$(wrd)' in topic $(current_topic) === ")
            TopicModels.addWord(lda, corpus, word_idx, current_topic)
            TopicModels.gibbsSampling(lda, corpus.documents, 20);
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
end

Add word 'triggered' in topic 8 === Add word 'graphics' in topic 7 === Add word 'windows' in topic 20 === Add word 'controller' in topic 10 === Add word 'apple' in topic 4 === Add word 'char' in topic 15 === Add word 'shipping' in topic 11 === Add word 'car' in topic 13 === Add word 'bike' in topic 2 === Add word 'reds' in topic 6 === Add word 'hockey' in topic 19 === Add word 'encryption' in topic 12 === Add word 'understandablemost' in topic 9 === Add word 'medical' in topic 16 === Add word 'space' in topic 18 === Add word 'god' in topic 1 === Add word 'gun' in topic 14 === Add word 'israeli' in topic 5 === Add word 'president' in topic 17 === Add word 'jesus' in topic 3 === 0.3932848847864422
Add word 'atheism' in topic 8 === Add word 'computer' in topic 7 === Add word 'unixxenix' in topic 20 === Add word 'drives' in topic 10 === Add word 'mac' in topic 4 === Add word 'window' in topic 15 === Add word 'waythey' in topic 11 === Add word 'cars' in topic 13 === Add word 'bikes' in topi

In [270]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
already_done_words = [];
t0 = time();
@time for loop in 1:5
    for current_cat in 1:20
        true_docs, absent_docs = docs_in_and_notin_cat(lda, corpus, current_cat)
        all_false_negatives = [i for i in 1:corpus.document_size if !(i in docs_ls[current_cat])]
        if absent_docs != []
            X, Y = make_PMI_dataset(absent_docs, all_false_negatives)
            word_idxs = sortperm(feature_selection.mutual_info_classif(X, Y), rev=true)[1:50]
            for i in word_idxs
                if !(i in already_done_words)
                    word_idx = i
                    push!(already_done_words, i)
                    break
                end
            end
            current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
            wrd = corpus.reverse_vocabulary[word_idx]
            print("Add word '$(wrd)' in topic $(current_topic) === ")
            TopicModels.addWord(lda, corpus, word_idx, current_topic)
            TopicModels.gibbsSampling(lda, corpus.documents, 20);
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
end
time() - t0

Add word 'oceania' in topic 8 === Add word 'graphics' in topic 7 === Add word 'windows' in topic 20 === Add word 'ide' in topic 10 === Add word 'apple' in topic 4 === Add word 'window' in topic 15 === Add word 'shipping' in topic 11 === Add word 'car' in topic 13 === Add word 'bike' in topic 2 === Add word 'cubs' in topic 6 === Add word 'hockey' in topic 19 === Add word 'encryption' in topic 12 === Add word 'alone' in topic 9 === Add word 'diet' in topic 16 === Add word 'space' in topic 18 === Add word 'god' in topic 1 === Add word 'gun' in topic 14 === Add word 'israeli' in topic 5 === Add word 'president' in topic 17 === Add word 'christian' in topic 3 === 0.39749908619344326
Add word 'atheism' in topic 8 === Add word 'computer' in topic 7 === Add word 'apps' in topic 20 === Add word 'controller' in topic 10 === Add word 'mac' in topic 4 === Add word 'widget' in topic 15 === Add word 'sale' in topic 11 === Add word 'cars' in topic 13 === Add word 'bikes' in topic 2 === Add word 'cinc

142725.4709019661

In [271]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
already_done_words = [];
t0 = time();
@time for loop in 1:5
    for current_cat in 1:20
        true_docs, absent_docs = docs_in_and_notin_cat(lda, corpus, current_cat)
        all_false_negatives = [i for i in 1:corpus.document_size if !(i in docs_ls[current_cat])]
        if absent_docs != []
            X, Y = make_PMI_dataset(absent_docs, all_false_negatives)
            word_idxs = sortperm(feature_selection.mutual_info_classif(X, Y), rev=true)[1:50]
            for i in word_idxs
                if !(i in already_done_words)
                    word_idx = i
                    push!(already_done_words, i)
                    break
                end
            end
            current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
            wrd = corpus.reverse_vocabulary[word_idx]
            print("Add word '$(wrd)' in topic $(current_topic) === ")
            TopicModels.addWord(lda, corpus, word_idx, current_topic)
            TopicModels.gibbsSampling(lda, corpus.documents, 20);
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
end
time() - t0

Add word 'altatheismmoderated' in topic 8 === Add word 'graphics' in topic 7 === Add word 'windows' in topic 20 === Add word 'drives' in topic 10 === Add word 'apple' in topic 4 === Add word 'window' in topic 15 === Add word 'shipping' in topic 11 === Add word 'car' in topic 13 === Add word 'bike' in topic 2 === Add word 'jays' in topic 6 === Add word 'hockey' in topic 19 === Add word 'encryption' in topic 12 === Add word 'sexmen' in topic 9 === Add word 'medical' in topic 16 === Add word 'space' in topic 18 === Add word 'god' in topic 1 === Add word 'gun' in topic 14 === Add word 'israeli' in topic 5 === Add word 'care' in topic 17 === Add word 'jesus' in topic 3 === 0.40015648990000696
Add word 'atheism' in topic 8 === Add word 'computer' in topic 7 === Add word 'applications' in topic 20 === Add word 'ide' in topic 10 === Add word 'mac' in topic 4 === Add word 'widget' in topic 15 === Add word 'jective' in topic 11 === Add word 'cars' in topic 13 === Add word 'motorcycle' in topic 2

134361.76042699814

In [272]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
already_done_words = [];
t0 = time();
@time for loop in 1:5
    for current_cat in 1:20
        true_docs, absent_docs = docs_in_and_notin_cat(lda, corpus, current_cat)
        all_false_negatives = [i for i in 1:corpus.document_size if !(i in docs_ls[current_cat])]
        if absent_docs != []
            X, Y = make_PMI_dataset(absent_docs, all_false_negatives)
            word_idxs = sortperm(feature_selection.mutual_info_classif(X, Y), rev=true)[1:50]
            for i in word_idxs
                if !(i in already_done_words)
                    word_idx = i
                    push!(already_done_words, i)
                    break
                end
            end
            current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
            wrd = corpus.reverse_vocabulary[word_idx]
            print("Add word '$(wrd)' in topic $(current_topic) === ")
            TopicModels.addWord(lda, corpus, word_idx, current_topic)
            TopicModels.gibbsSampling(lda, corpus.documents, 20);
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
end
time() - t0

Add word 'hypothesis' in topic 8 === Add word 'graphics' in topic 7 === Add word 'windows' in topic 20 === Add word 'ide' in topic 10 === Add word 'apple' in topic 4 === Add word 'function' in topic 15 === Add word 'shipping' in topic 11 === Add word 'car' in topic 13 === Add word 'bike' in topic 2 === Add word 'reds' in topic 6 === Add word 'hockey' in topic 19 === Add word 'encryption' in topic 12 === Add word 'washington' in topic 9 === Add word 'medical' in topic 16 === Add word 'space' in topic 18 === Add word 'god' in topic 1 === Add word 'gun' in topic 14 === Add word 'israeli' in topic 5 === Add word 'taxes' in topic 17 === Add word 'christian' in topic 3 === 0.39750901038836106
Add word 'atheism' in topic 8 === Add word 'animation' in topic 7 === Add word 'dos' in topic 20 === Add word 'controller' in topic 10 === Add word 'mac' in topic 4 === Add word 'window' in topic 15 === Add word 'cod' in topic 11 === Add word 'engine' in topic 13 === Add word 'bikes' in topic 2 === Add 

144033.13406205177

In [273]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
already_done_words = [];
t0 = time();
@time for loop in 1:5
    for current_cat in 1:20
        true_docs, absent_docs = docs_in_and_notin_cat(lda, corpus, current_cat)
        all_false_negatives = [i for i in 1:corpus.document_size if !(i in docs_ls[current_cat])]
        if absent_docs != []
            X, Y = make_PMI_dataset(absent_docs, all_false_negatives)
            word_idxs = sortperm(feature_selection.mutual_info_classif(X, Y), rev=true)[1:50]
            for i in word_idxs
                if !(i in already_done_words)
                    word_idx = i
                    push!(already_done_words, i)
                    break
                end
            end
            current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
            wrd = corpus.reverse_vocabulary[word_idx]
            print("Add word '$(wrd)' in topic $(current_topic) === ")
            TopicModels.addWord(lda, corpus, word_idx, current_topic)
            TopicModels.gibbsSampling(lda, corpus.documents, 20);
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
end
time() - t0

Add word 'resumes' in topic 8 === Add word 'graphics' in topic 7 === Add word 'windows' in topic 20 === Add word 'controller' in topic 10 === Add word 'apple' in topic 4 === Add word 'char' in topic 15 === Add word 'shipping' in topic 11 === Add word 'car' in topic 13 === Add word 'bike' in topic 2 === Add word 'braves' in topic 6 === Add word 'team' in topic 19 === Add word 'encryption' in topic 12 === Add word 'circuit' in topic 9 === Add word 'medical' in topic 16 === Add word 'space' in topic 18 === Add word 'god' in topic 1 === Add word 'gun' in topic 14 === Add word 'israeli' in topic 5 === Add word 'house' in topic 17 === Add word 'paidophthoreseis' in topic 3 === 0.4065116551879326
Add word 'atheism' in topic 8 === Add word 'image' in topic 7 === Add word 'dos' in topic 20 === Add word 'ide' in topic 10 === Add word 'mac' in topic 4 === Add word 'exit' in topic 15 === Add word 'instantiated' in topic 11 === Add word 'cars' in topic 13 === Add word 'motorcycle' in topic 2 === Ad

156098.35910701752

In [274]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
already_done_words = [];
t0 = time();
@time for loop in 1:5
    for current_cat in 1:20
        true_docs, absent_docs = docs_in_and_notin_cat(lda, corpus, current_cat)
        all_false_negatives = [i for i in 1:corpus.document_size if !(i in docs_ls[current_cat])]
        if absent_docs != []
            X, Y = make_PMI_dataset(absent_docs, all_false_negatives)
            word_idxs = sortperm(feature_selection.mutual_info_classif(X, Y), rev=true)[1:50]
            for i in word_idxs
                if !(i in already_done_words)
                    word_idx = i
                    push!(already_done_words, i)
                    break
                end
            end
            current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
            wrd = corpus.reverse_vocabulary[word_idx]
            print("Add word '$(wrd)' in topic $(current_topic) === ")
            TopicModels.addWord(lda, corpus, word_idx, current_topic)
            TopicModels.gibbsSampling(lda, corpus.documents, 20);
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
end
time() - t0

Add word 'book' in topic 8 === Add word 'graphics' in topic 7 === Add word 'windows' in topic 20 === Add word 'ide' in topic 10 === Add word 'mac' in topic 4 === Add word 'char' in topic 15 === Add word 'shipping' in topic 11 === Add word 'car' in topic 13 === Add word 'bike' in topic 2 === Add word 'marlins' in topic 6 === Add word 'hockey' in topic 19 === Add word 'encryption' in topic 12 === Add word 'lovely' in topic 9 === Add word 'medical' in topic 16 === Add word 'space' in topic 18 === Add word 'god' in topic 1 === Add word 'gun' in topic 14 === Add word 'israeli' in topic 5 === Add word 'economic' in topic 17 === Add word 'jesus' in topic 3 === 0.3926946200601344
Add word 'atheism' in topic 8 === Add word 'fax' in topic 7 === Add word 'applications' in topic 20 === Add word 'drive' in topic 10 === Add word 'apple' in topic 4 === Add word 'window' in topic 15 === Add word 'sale' in topic 11 === Add word 'cars' in topic 13 === Add word 'bikes' in topic 2 === Add word 'jays' in t

144362.32349085808

In [275]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
already_done_words = [];
t0 = time();
@time for loop in 1:5
    for current_cat in 1:20
        true_docs, absent_docs = docs_in_and_notin_cat(lda, corpus, current_cat)
        all_false_negatives = [i for i in 1:corpus.document_size if !(i in docs_ls[current_cat])]
        if absent_docs != []
            X, Y = make_PMI_dataset(absent_docs, all_false_negatives)
            word_idxs = sortperm(feature_selection.mutual_info_classif(X, Y), rev=true)[1:50]
            for i in word_idxs
                if !(i in already_done_words)
                    word_idx = i
                    push!(already_done_words, i)
                    break
                end
            end
            current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
            wrd = corpus.reverse_vocabulary[word_idx]
            print("Add word '$(wrd)' in topic $(current_topic) === ")
            TopicModels.addWord(lda, corpus, word_idx, current_topic)
            TopicModels.gibbsSampling(lda, corpus.documents, 20);
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
end
time() - t0

Add word 'mapssolar' in topic 8 === Add word 'graphics' in topic 7 === Add word 'windows' in topic 20 === Add word 'ide' in topic 10 === Add word 'apple' in topic 4 === Add word 'judgestoadcom' in topic 15 === Add word 'shipping' in topic 11 === Add word 'car' in topic 13 === Add word 'bike' in topic 2 === Add word 'cubs' in topic 6 === Add word 'hockey' in topic 19 === Add word 'encryption' in topic 12 === Add word 'insult' in topic 9 === Add word 'medical' in topic 16 === Add word 'space' in topic 18 === Add word 'god' in topic 1 === Add word 'gun' in topic 14 === Add word 'israeli' in topic 5 === Add word 'economic' in topic 17 === Add word 'bible' in topic 3 === 0.40247930225341677
Add word 'atheism' in topic 8 === Add word 'computer' in topic 7 === Add word 'dos' in topic 20 === Add word 'drive' in topic 10 === Add word 'mac' in topic 4 === Add word 'window' in topic 15 === Add word 'cod' in topic 11 === Add word 'cars' in topic 13 === Add word 'motorcycle' in topic 2 === Add word

179257.42425704002

In [276]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
already_done_words = [];
t0 = time();
@time for loop in 1:5
    for current_cat in 1:20
        true_docs, absent_docs = docs_in_and_notin_cat(lda, corpus, current_cat)
        all_false_negatives = [i for i in 1:corpus.document_size if !(i in docs_ls[current_cat])]
        if absent_docs != []
            X, Y = make_PMI_dataset(absent_docs, all_false_negatives)
            word_idxs = sortperm(feature_selection.mutual_info_classif(X, Y), rev=true)[1:50]
            for i in word_idxs
                if !(i in already_done_words)
                    word_idx = i
                    push!(already_done_words, i)
                    break
                end
            end
            current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
            wrd = corpus.reverse_vocabulary[word_idx]
            print("Add word '$(wrd)' in topic $(current_topic) === ")
            TopicModels.addWord(lda, corpus, word_idx, current_topic)
            TopicModels.gibbsSampling(lda, corpus.documents, 20);
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
end
time() - t0

Add word 'injectionstyle' in topic 8 === Add word 'graphics' in topic 7 === Add word 'windows' in topic 20 === Add word 'ide' in topic 10 === Add word 'apple' in topic 4 === Add word 'char' in topic 15 === Add word 'cod' in topic 11 === Add word 'car' in topic 13 === Add word 'bike' in topic 2 === Add word 'reds' in topic 6 === Add word 'hockey' in topic 19 === Add word 'encryption' in topic 12 === Add word 'calsnow' in topic 9 === Add word 'medical' in topic 16 === Add word 'space' in topic 18 === Add word 'god' in topic 1 === Add word 'gun' in topic 14 === Add word 'israeli' in topic 5 === Add word 'president' in topic 17 === Add word 'jesus' in topic 3 === 0.3908968075803419
Add word 'atheism' in topic 8 === Add word 'computational' in topic 7 === Add word 'dodging' in topic 20 === Add word 'drives' in topic 10 === Add word 'mac' in topic 4 === Add word 'widgets' in topic 15 === Add word 'shipping' in topic 11 === Add word 'cars' in topic 13 === Add word 'bikes' in topic 2 === Add w

210512.29003691673

In [277]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
already_done_words = [];
t0 = time();
@time for loop in 1:5
    for current_cat in 1:20
        true_docs, absent_docs = docs_in_and_notin_cat(lda, corpus, current_cat)
        all_false_negatives = [i for i in 1:corpus.document_size if !(i in docs_ls[current_cat])]
        if absent_docs != []
            X, Y = make_PMI_dataset(absent_docs, all_false_negatives)
            word_idxs = sortperm(feature_selection.mutual_info_classif(X, Y), rev=true)[1:50]
            for i in word_idxs
                if !(i in already_done_words)
                    word_idx = i
                    push!(already_done_words, i)
                    break
                end
            end
            current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
            wrd = corpus.reverse_vocabulary[word_idx]
            print("Add word '$(wrd)' in topic $(current_topic) === ")
            TopicModels.addWord(lda, corpus, word_idx, current_topic)
            TopicModels.gibbsSampling(lda, corpus.documents, 20);
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
end
time() - t0

Add word 'jbig' in topic 8 === Add word 'graphics' in topic 7 === Add word 'windows' in topic 20 === Add word 'ide' in topic 10 === Add word 'mac' in topic 4 === Add word 'window' in topic 15 === Add word 'shipping' in topic 11 === Add word 'car' in topic 13 === Add word 'bike' in topic 2 === Add word 'jays' in topic 6 === Add word 'hockey' in topic 19 === Add word 'encryption' in topic 12 === Add word 'circuit' in topic 9 === Add word 'medical' in topic 16 === Add word 'space' in topic 18 === Add word 'god' in topic 1 === Add word 'gun' in topic 14 === Add word 'israeli' in topic 5 === Add word 'reform' in topic 17 === Add word 'jesus' in topic 3 === 0.3953775497555705
Add word 'atheism' in topic 8 === Add word 'animation' in topic 7 === Add word 'dos' in topic 20 === Add word 'drives' in topic 10 === Add word 'apple' in topic 4 === Add word 'widget' in topic 15 === Add word 'beads' in topic 11 === Add word 'engine' in topic 13 === Add word 'ride' in topic 2 === Add word 'marlins' in 

260593.42877984047

In [125]:
function make_PMI_dataset(absent_docs, all_false_negatives)
    X = []; Y = []
    for i in absent_docs
        push!(X, position_to_feature(i))
        push!(Y, 1)
    end
    for i in all_false_negatives
        push!(X, position_to_feature(i))
        push!(Y, 0)
    end
    return X, Y
end

make_PMI_dataset (generic function with 1 method)

In [80]:
function position_to_feature(doc_idx)
    position_vector = corpus.documents[doc_idx]
    feature_vector = [0 for i in 1:corpus.vocab_count]
    for (idx, val) in enumerate(position_vector)
        feature_vector[val] +=1
    end
    return feature_vector
end

position_to_feature (generic function with 1 method)

In [101]:
function docs_in_and_notin_cat(lda, corpus, cat_idx)
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    topic_x_top_docs = [idx for (idx, val) in enumerate(predicted_with_marriage_assigment) if val==cat_idx];
    docs_in_cat_notin_topic = []
    for i in docs_ls[cat_idx]
        if !(i in topic_x_top_docs)
            push!(docs_in_cat_notin_topic, i)
        end
    end
    docs_in_cat_in_topic = [i for i in docs_ls[cat_idx] if !(i in docs_in_cat_notin_topic)]
    return docs_in_cat_in_topic, docs_in_cat_notin_topic
end

docs_in_and_notin_cat (generic function with 1 method)

In [54]:
function topic_of_each_doc(lda, corpus)
    top_topic_for_each_doc = []
    for i in 1:corpus.document_size
        top_val = 0
        top_topic = 0
        for j in 1:lda.M
            v = TopicModels.topicPredict(lda, i, j)
            if v>=top_val
                top_val = v
                top_topic = j
            end
        end
        push!(top_topic_for_each_doc, top_topic)
    end
    return top_topic_for_each_doc
end

topic_of_each_doc (generic function with 1 method)

## Remove_document refinement after stable marriage algorithm

In [26]:
true_cat_original = [17, 7, 18, 17, 10, 19, 16, 19, 18, 8, 13, 7, 12, 9, 18, 18, 17, 17, 15, 17, 7, 1, 19, 5, 4, 8, 19, 17, 15, 15, 17, 0, 19, 11, 17, 18, 9, 13, 19, 17, 13, 17, 19, 15, 7, 8, 11, 9, 1, 8, 0, 18, 17, 14, 14, 10, 15, 3, 8, 10, 10, 11, 17, 6, 17, 17, 15, 8, 15, 11, 2, 17, 9, 18, 17, 17, 10, 2, 9, 1, 10, 4, 9, 15, 3, 15, 19, 15, 16, 13, 17, 11, 0, 18, 10, 14, 6, 17, 15, 17, 19, 14, 8, 14, 5, 5, 11, 15, 6, 10, 5, 9, 15, 11, 9, 15, 7, 1, 14, 16, 15, 13, 13, 17, 0, 11, 16, 3, 16, 15, 14, 14, 19, 9, 18, 14, 17, 17, 4, 17, 15, 1, 16, 11, 15, 6, 2, 14, 15, 18, 8, 17, 18, 4, 7, 11, 17, 18, 9, 16, 18, 7, 11, 2, 15, 19, 1, 11, 14, 17, 1, 16, 6, 5, 19, 7, 11, 16, 8, 16, 10, 15, 17, 16, 16, 1, 0, 7, 15, 0, 6, 18, 10, 9, 11, 16, 19, 14, 6, 0, 0, 3, 17, 17, 12, 16, 8, 6, 19, 10, 17, 16, 14, 4, 16, 11, 1, 14, 0, 2, 16, 15, 15, 6, 15, 0, 14, 10, 15, 13, 2, 19, 15, 17, 15, 1, 9, 16, 18, 17, 6, 17, 8, 14, 11, 5, 15, 7, 15, 15, 12, 11, 14, 14, 3, 15, 3, 17, 3, 14, 11, 19, 10, 18, 15, 16, 15, 15, 9, 16, 11, 4, 19, 13, 3, 19, 9, 17, 18, 7, 7, 15, 16, 14, 15, 10, 5, 9, 14, 14, 17, 15, 15, 19, 14, 15, 15, 0, 4, 11, 11, 15, 12, 6, 3, 15, 14, 5, 14, 18, 11, 11, 14, 17, 17, 10, 9, 5, 7, 10, 19, 18, 17, 15, 18, 17, 11, 11, 13, 3, 2, 15, 10, 17, 2, 14, 13, 15, 15, 16, 15, 13, 14, 7, 16, 19, 3, 17, 17, 17, 14, 16, 9, 8, 14, 14, 16, 13, 12, 11, 17, 11, 1, 4, 11, 18, 16, 15, 11, 17, 15, 6, 17, 17, 13, 9, 10, 18, 17, 17, 13, 6, 1, 9, 14, 19, 3, 19, 13, 14, 10, 16, 19, 14, 9, 17, 7, 7, 11, 17, 18, 11, 17, 11, 18, 17, 16, 14, 1, 11, 16, 11, 18, 6, 1, 14, 15, 6, 17, 18, 16, 6, 13, 11, 0, 0, 16, 8, 17, 17, 10, 4, 15, 15, 15, 1, 13, 15, 14, 5, 15, 9, 1, 0, 13, 11, 1, 14, 16, 7, 2, 6, 10, 8, 14, 12, 19, 2, 7, 15, 17, 13, 17, 13, 17, 17, 19, 16, 15, 0, 12, 19, 19, 18, 15, 16, 9, 15, 16, 16, 12, 9, 9, 18, 4, 18, 11, 17, 15, 12, 12, 17, 17, 15, 19, 15, 19, 17, 15, 10, 14, 19, 16, 17, 10, 10, 14, 16, 5, 13, 9, 15, 17, 15, 19, 0, 18, 14, 15, 1, 15, 5, 17, 12, 18, 10, 5, 16, 0, 15, 1, 5, 7, 11, 19, 13, 11, 17, 9, 18, 9, 4, 10, 11, 5, 15, 12, 15, 1, 18, 6, 13, 18, 9, 1, 17, 18, 17, 15, 15, 19, 3, 17, 15, 9, 11, 14, 17, 14, 15, 1, 15, 15, 10, 3, 19, 17, 14, 6, 11, 12, 15, 17, 15, 13, 3, 0, 10, 4, 11, 0, 14, 7, 0, 15, 0, 2, 17, 17, 6, 19, 14, 17, 14, 6, 15, 12, 13, 4, 1, 17, 16, 17, 13, 18, 0, 11, 10, 3, 0, 13, 15, 13, 15, 15, 13, 5, 18, 1, 11, 19, 15, 13, 13, 10, 17, 13, 14, 19, 14, 0, 5, 7, 15, 11, 1, 15, 15, 14, 15, 13, 0, 15, 18, 7, 8, 13, 11, 16, 14, 19, 15, 16, 0, 4, 17, 11, 10, 11, 5, 19, 10, 17, 7, 10, 17, 9, 11, 15, 19, 17, 11, 11, 11, 10, 16, 14, 17, 18, 13, 16, 19, 9, 11, 18, 16, 1, 13, 16, 18, 16, 11, 16, 8, 14, 14, 16, 15, 13, 15, 11, 11, 16, 2, 15, 16, 19, 8, 17, 19, 0, 16, 15, 14, 17, 9, 19, 6, 10, 16, 15, 0, 17, 10, 6, 16, 16, 17, 11, 15, 14, 10, 2, 18, 0, 18, 16, 10, 8, 3, 17, 17, 15, 11, 17, 13, 9, 18, 19, 11, 17, 11, 11, 5, 18, 17, 13, 11, 14, 13, 17, 15, 2, 0, 12, 0, 0, 14, 15, 8, 18, 9, 17, 10, 15, 13, 13, 3, 4, 16, 10, 5, 2, 2, 16, 17, 15, 7, 15, 17, 16, 12, 17, 15, 16, 15, 17, 17, 17, 18, 19, 17, 14, 18, 11, 12, 18, 6, 14, 11, 0, 15, 10, 17, 3, 11, 9, 15, 11, 9, 13, 16, 15, 13, 17, 18, 10, 4, 18, 19, 5, 19, 17, 11, 10, 19, 17, 19, 18, 10, 3, 13, 10, 16, 11, 5, 0, 3, 11, 17, 13, 14, 6, 16, 4, 19, 19, 14, 1, 14, 16, 11, 6, 18, 19, 11, 10, 11, 19, 11, 9, 19, 10, 17, 10, 18, 14, 11, 9, 0, 9, 1, 15, 11, 8, 14, 18, 11, 16, 15, 16, 6, 18, 13, 18, 2, 17, 15, 16, 15, 11, 16, 5, 15, 11, 14, 1, 16, 15, 11, 19, 16, 11, 15, 17, 15, 2, 16, 11, 7, 13, 17, 16, 7, 10, 1, 16, 1, 18, 17, 11, 17, 11, 5, 17, 1, 18, 0, 18, 16, 17, 12, 17, 11, 11, 15, 15, 12, 0, 19, 10, 17, 14, 12, 0, 19, 11, 2, 16, 19, 17, 15, 10, 5, 16, 5, 5, 13, 11, 19, 4, 15, 14, 11, 17, 15, 14, 8, 17, 3, 13, 0, 17, 0, 10, 12, 14, 6, 13, 13, 11, 10, 13, 10, 14, 0, 19, 11, 7, 10, 14, 3, 15, 3, 14, 10, 17, 4, 5, 11, 0, 19, 10, 10, 4, 1, 13, 17, 7, 5, 19, 13, 11, 14, 19, 19, 18, 2, 13, 6, 9, 0, 11, 11, 6, 11, 18, 16, 14, 10, 2, 8, 16, 5, 17, 16, 13, 18, 17, 13, 18, 15, 19, 6, 18, 17, 19, 12, 18, 9, 13, 6, 0, 15, 11, 18, 11, 0, 18, 15, 10, 10, 6, 5, 17, 11, 1, 9, 1, 14, 12, 14, 10, 15, 11, 11, 18, 2, 10, 3, 17, 18, 14, 19, 17, 15, 9, 5, 1, 9, 13, 15, 13, 10, 17, 19, 4, 17, 17, 15, 10, 17, 13, 18, 17, 11, 18, 18, 15, 15, 17, 11, 10, 8, 0, 15, 15, 14, 11, 15, 11, 8, 15, 3, 12, 8, 15, 14, 15, 10, 10, 11, 11, 3, 15, 9, 19, 17, 4, 7, 15, 4, 9, 9, 1, 6, 13, 10, 19, 9, 11, 16, 19, 17, 16, 14, 10, 9, 15, 15, 9, 5, 0, 2, 18, 9, 15, 19, 19, 17, 5, 10, 14, 15, 17, 15, 18, 17, 15, 4, 8, 13, 8, 11, 1, 15, 13, 15, 9, 9, 17, 14, 10, 14, 0, 15, 0, 17, 10, 0, 17, 0, 10, 14, 0, 12, 16, 17, 13, 13, 19, 3, 15, 0, 16, 19, 11, 14, 17, 19, 0, 14, 2, 18, 17, 15, 2, 17, 3, 11, 14, 9, 4, 3, 9, 0, 11, 13, 5, 15, 10, 14, 10, 15, 0, 9, 17, 13, 0, 5, 17, 16, 7, 0, 19, 15, 18, 15, 17, 0, 1, 15, 11, 17, 15, 9, 18, 6, 11, 17, 12, 13, 15, 18, 7, 17, 15, 15, 6, 13, 11, 18, 4, 5, 13, 7, 17, 18, 17, 15, 18, 0, 6, 5, 18, 13, 14, 2, 19, 8, 8, 10, 8, 17, 11, 15, 10, 17, 0, 19, 8, 6, 15, 12, 15, 16, 12, 19, 6, 6, 9, 14, 14, 17, 16, 16, 13, 6, 13, 15, 6, 15, 0, 10, 10, 15, 13, 14, 15, 17, 10, 15, 17, 11, 5, 9, 12, 16, 10, 13, 16, 2, 17, 6, 5, 0, 15, 9, 15, 19, 0, 11, 18, 1, 3, 18, 18, 18, 17, 0, 4, 17, 15, 15, 17, 1, 1, 15, 1, 15, 15, 16, 19, 17, 15, 2, 17, 15, 11, 16, 11, 15, 12, 15, 6, 13, 16, 19, 3, 5, 5, 7, 6, 1, 1, 16, 15, 18, 0, 15, 0, 15, 17, 19, 15, 0, 16, 14, 17, 6, 7, 15, 11, 12, 15, 15, 17, 15, 12, 17, 1, 1, 14, 3, 6, 17, 17, 18, 15, 1, 6, 10, 18, 10, 2, 11, 13, 17, 16, 3, 17, 19, 19, 18, 15, 8, 5, 16, 11, 0, 9, 13, 0, 9, 10, 1, 15, 1, 10, 17, 1, 14, 16, 1, 14, 15, 11, 14, 6, 8, 15, 9, 19, 16, 7, 17, 14, 17, 15, 18, 5, 11, 15, 2, 16, 5, 15, 9, 12, 17, 10, 15, 17, 19, 9, 1, 15, 17, 18, 3, 16, 18, 1, 13, 3, 18, 18, 15, 17, 17, 9, 19, 3, 11, 16, 17, 10, 1, 7, 19, 9, 19, 1, 17, 10, 18, 17, 8, 10, 17, 19, 13, 5, 18, 15, 11, 19, 0, 14, 11, 15, 15, 18, 18, 16, 13, 2, 16, 12, 5, 6, 0, 5, 15, 3, 16, 8, 8, 1, 17, 13, 18, 17, 17, 14, 6, 16, 14, 4, 3, 3, 3, 17, 7, 4, 14, 15, 18, 11, 16, 17, 15, 17, 6, 9, 13, 17, 6, 17, 18, 3, 17, 15, 3, 15, 18, 10, 17, 2, 18, 11, 13, 11, 15, 1, 10, 18, 19, 14, 11, 18, 15, 0, 1, 15, 17, 15, 8, 11, 18, 1, 7, 19, 10, 0, 9, 17, 9, 17, 11, 1, 19, 7, 15, 13, 17, 17, 0, 9, 16, 18, 18, 13, 15, 5, 7, 15, 15, 4, 15, 5, 5, 8, 9, 11, 13, 12, 16, 17, 17, 1, 19, 15, 14, 13, 11, 13, 13, 13, 14, 2, 9, 15, 7, 15, 2, 8, 11, 15, 11, 0, 4, 11, 15, 0, 15, 1, 5, 17, 5, 11, 5, 4, 8, 17, 3, 5, 14, 5, 13, 10, 16, 17, 9, 7, 6, 10, 10, 19, 3, 14, 17, 13, 1, 16, 0, 7, 17, 15, 0, 11, 15, 19, 16, 18, 0, 10, 17, 1, 17, 1, 15, 15, 9, 12, 14, 15, 11, 10, 11, 16, 13, 17, 15, 2, 13, 19, 19, 15, 16, 10, 19, 6, 14, 17, 13, 19, 15, 6, 17, 16, 15, 17, 18, 7, 12, 14, 17, 13, 1, 5, 9, 0, 6, 4, 17, 4, 13, 17, 16, 19, 1, 6, 14, 11, 11, 9, 14, 10, 16, 3, 12, 17, 15, 17, 1, 3, 13, 14, 10, 5, 14, 15, 6, 10, 15, 16, 14, 15, 15, 9, 1, 17, 15, 19, 17, 2, 9, 9, 1, 15, 17, 17, 15, 16, 16, 14, 9, 17, 17, 10, 4, 15, 17, 16, 18, 18, 19, 17, 3, 12, 7, 11, 3, 13, 12, 15, 1, 19, 17, 12, 16, 9, 8, 14, 0, 16, 5, 10, 15, 15, 15, 12, 17, 18, 15, 14, 2, 15, 18, 0, 15, 4, 1, 1, 1, 13, 13, 3, 16, 9, 19, 17, 12, 10, 18, 0, 0, 14, 7, 13, 14, 17, 13, 17, 1, 6, 9, 15, 4, 15, 8, 17, 18, 7, 19, 16, 9, 6, 16, 10, 14, 6, 18, 18, 11, 17, 7, 18, 13, 3, 17, 17, 15, 19, 19, 11, 0, 17, 15, 19, 18, 15, 5, 10, 0, 0, 0, 1, 13, 0, 12, 18, 8, 0, 2, 15, 17, 18, 6, 17, 13, 15, 0, 17, 18, 15, 18, 13, 19, 10, 8, 15, 2, 1, 15, 17, 15, 17, 17]
docs_cat = [i+1 for i in true_cat_original];
docs_ls = [[] for i in 1:20]
for (idx, val) in enumerate(docs_cat)
    push!(docs_ls[val], idx)
end

In [27]:
topic_marriage_assigment = Dict(1=>16, 2=>9, 3=>20, 4=>5, 5=>18, 6=>10, 7=>2, 8=>1, 9=>13, 10=>4, 
                            11=>7, 12=>12, 13=>8, 14=>17, 15=>6, 16=>14, 17=>19, 18=>15, 19=>11, 20=>3);

In [34]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");

In [35]:
already_done_docs = []

Any[]

In [40]:
current_cat = 2

2

In [42]:
false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)

6-element Array{Any,1}:
    6
   64
  666
 1091
 1108
 1847

In [43]:
current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]

7

In [45]:
TopicModels.removeDoc(lda, corpus, 6, current_topic)

In [46]:
TopicModels.gibbsSampling(lda, corpus.documents, 20);

In [47]:
top_topic_for_each_doc = topic_of_each_doc(lda, corpus);

In [49]:
predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
print(predicted_with_marriage_assigment)

[9, 10, 19, 1, 11, 1, 19, 16, 19, 8, 14, 1, 1, 10, 14, 17, 18, 16, 16, 18, 8, 1, 16, 6, 7, 1, 16, 5, 1, 1, 18, 1, 1, 12, 1, 19, 10, 14, 1, 11, 13, 19, 16, 16, 1, 10, 4, 10, 6, 1, 1, 17, 18, 15, 15, 1, 20, 7, 10, 11, 11, 12, 9, 2, 18, 19, 16, 1, 1, 12, 6, 18, 10, 1, 1, 18, 13, 6, 1, 2, 10, 8, 11, 16, 1, 16, 16, 16, 5, 13, 1, 12, 1, 1, 1, 15, 6, 19, 1, 18, 16, 15, 9, 15, 4, 6, 1, 1, 7, 11, 4, 11, 16, 1, 10, 16, 10, 6, 15, 17, 1, 1, 1, 18, 1, 8, 1, 7, 17, 16, 8, 15, 1, 10, 5, 15, 18, 18, 7, 18, 1, 6, 17, 12, 16, 7, 7, 8, 1, 17, 10, 1, 1, 7, 1, 1, 1, 1, 10, 1, 1, 8, 12, 6, 1, 1, 6, 12, 15, 18, 1, 17, 6, 4, 20, 10, 12, 19, 1, 9, 10, 16, 18, 1, 5, 6, 1, 8, 1, 16, 7, 8, 10, 10, 12, 17, 5, 15, 8, 19, 1, 7, 18, 18, 14, 1, 10, 8, 1, 11, 19, 1, 15, 7, 19, 12, 3, 15, 1, 6, 10, 16, 16, 7, 1, 1, 1, 1, 16, 5, 6, 16, 1, 18, 16, 6, 10, 19, 19, 18, 7, 18, 10, 15, 12, 4, 1, 1, 16, 1, 4, 12, 8, 15, 1, 16, 7, 18, 7, 1, 1, 1, 11, 14, 1, 17, 16, 1, 10, 1, 12, 7, 1, 1, 6, 9, 10, 19, 1, 1, 1, 9, 19, 15, 16, 11

In [50]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
already_done_docs = []
@time for loop in 1:5
    for current_cat in 1:20
        false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
        if false_docs_for_cat != []
            for i in false_docs_for_cat
                if !(i in already_done_docs)
                    current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
                    TopicModels.removeDoc(lda, corpus, i, current_topic)
                    TopicModels.gibbsSampling(lda, corpus.documents, 20);
                    push!(already_done_docs, i)
                    break
                end
            end
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    print(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
    print(", ")
end

0.39971314050362267, 0.3811414733709356, 0.38871513138412755, 0.4074351733065137, 0.4085268669490589, 11535.291230 seconds (160.65 G allocations: 2.692 TiB, 8.69% gc time)


In [51]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
already_done_docs = []
@time for loop in 1:5
    for current_cat in 1:20
        false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
        if false_docs_for_cat != []
            for i in false_docs_for_cat
                if !(i in already_done_docs)
                    current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
                    TopicModels.removeDoc(lda, corpus, i, current_topic)
                    TopicModels.gibbsSampling(lda, corpus.documents, 20);
                    push!(already_done_docs, i)
                    break
                end
            end
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    print(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
    print(", ")
end

0.38352985972247033, 0.38206773523990567, 0.3767813265594643, 0.3860958960217968, 0.3892384042053968, 10493.513903 seconds (162.41 G allocations: 2.721 TiB, 8.37% gc time)


In [52]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
already_done_docs = []
@time for loop in 1:5
    for current_cat in 1:20
        false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
        if false_docs_for_cat != []
            for i in false_docs_for_cat
                if !(i in already_done_docs)
                    current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
                    TopicModels.removeDoc(lda, corpus, i, current_topic)
                    TopicModels.gibbsSampling(lda, corpus.documents, 20);
                    push!(already_done_docs, i)
                    break
                end
            end
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    print(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
    print(", ")
end

0.3926436033005589, 0.37625135549017746, 0.3785444706186156, 0.3718701565424993, 0.3759927339991509, 10216.785187 seconds (158.93 G allocations: 2.663 TiB, 9.00% gc time)


In [53]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
already_done_docs = []
@time for loop in 1:5
    for current_cat in 1:20
        false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
        if false_docs_for_cat != []
            for i in false_docs_for_cat
                if !(i in already_done_docs)
                    current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
                    TopicModels.removeDoc(lda, corpus, i, current_topic)
                    TopicModels.gibbsSampling(lda, corpus.documents, 20);
                    push!(already_done_docs, i)
                    break
                end
            end
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    print(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
    print(", ")
end

0.3832850816953736, 0.3652210312637861, 0.37220127093713723, 0.37853255863213975, 0.390190906453969, 11384.071323 seconds (164.10 G allocations: 2.750 TiB, 8.00% gc time)


In [54]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
already_done_docs = []
@time for loop in 1:5
    for current_cat in 1:20
        false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
        if false_docs_for_cat != []
            for i in false_docs_for_cat
                if !(i in already_done_docs)
                    current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
                    TopicModels.removeDoc(lda, corpus, i, current_topic)
                    TopicModels.gibbsSampling(lda, corpus.documents, 20);
                    push!(already_done_docs, i)
                    break
                end
            end
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    print(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
    print(", ")
end

0.3910496282633884, 0.38841475060779806, 0.3928616722019845, 0.3934339934643961, 0.399411773064334, 11093.017515 seconds (162.38 G allocations: 2.721 TiB, 8.95% gc time)


In [55]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
already_done_docs = []
@time for loop in 1:5
    for current_cat in 1:20
        false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
        if false_docs_for_cat != []
            for i in false_docs_for_cat
                if !(i in already_done_docs)
                    current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
                    TopicModels.removeDoc(lda, corpus, i, current_topic)
                    TopicModels.gibbsSampling(lda, corpus.documents, 20);
                    push!(already_done_docs, i)
                    break
                end
            end
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    print(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
    print(", ")
end

0.4011463228979381, 0.39096569020994654, 0.3938450727833862, 0.4085129069940024, 0.4158062754067186, 10393.455148 seconds (158.91 G allocations: 2.663 TiB, 8.24% gc time)


In [56]:
#7th iteration
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
already_done_docs = []
@time for loop in 1:5
    for current_cat in 1:20
        false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
        if false_docs_for_cat != []
            for i in false_docs_for_cat
                if !(i in already_done_docs)
                    current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
                    TopicModels.removeDoc(lda, corpus, i, current_topic)
                    TopicModels.gibbsSampling(lda, corpus.documents, 20);
                    push!(already_done_docs, i)
                    break
                end
            end
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    print(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
    print(", ")
end

0.3909799004389126, 0.37566707499716767, 0.3839828677274133, 0.39190921026851766, 0.4035818346037484, 10673.713715 seconds (162.38 G allocations: 2.721 TiB, 9.01% gc time)


In [57]:
# 8th iteration
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
already_done_docs = []
@time for loop in 1:5
    for current_cat in 1:20
        false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
        if false_docs_for_cat != []
            for i in false_docs_for_cat
                if !(i in already_done_docs)
                    current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
                    TopicModels.removeDoc(lda, corpus, i, current_topic)
                    TopicModels.gibbsSampling(lda, corpus.documents, 20);
                    push!(already_done_docs, i)
                    break
                end
            end
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    print(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
    print(", ")
end

0.39178711525039284, 0.3730444630265938, 0.3797254324703822, 0.38723200513414835, 0.39000128734636574, 10932.916948 seconds (160.65 G allocations: 2.692 TiB, 8.61% gc time)


In [58]:
# 9th iteration
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
already_done_docs = []
@time for loop in 1:5
    for current_cat in 1:20
        false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
        if false_docs_for_cat != []
            for i in false_docs_for_cat
                if !(i in already_done_docs)
                    current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
                    TopicModels.removeDoc(lda, corpus, i, current_topic)
                    TopicModels.gibbsSampling(lda, corpus.documents, 20);
                    push!(already_done_docs, i)
                    break
                end
            end
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    print(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
    print(", ")
end

0.3889407200505525, 0.3756896025923708, 0.37995306226815645, 0.38817729262408635, 0.39861900850553117, 10742.241071 seconds (160.67 G allocations: 2.692 TiB, 9.10% gc time)


In [59]:
# 10th iteration
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
already_done_docs = []
@time for loop in 1:5
    for current_cat in 1:20
        false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
        if false_docs_for_cat != []
            for i in false_docs_for_cat
                if !(i in already_done_docs)
                    current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
                    TopicModels.removeDoc(lda, corpus, i, current_topic)
                    TopicModels.gibbsSampling(lda, corpus.documents, 20);
                    push!(already_done_docs, i)
                    break
                end
            end
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    print(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
    print(", ")
end

0.3947284835022867, 0.3942943598402001, 0.40388718244051025, 0.4099690066204232, 0.40881690603327653, 10182.828959 seconds (158.86 G allocations: 2.662 TiB, 8.44% gc time)


In [30]:
function false_docs_in_cat(lda, corpus, docs_ls, cat_idx)
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    topic_x_top_docs = [idx for (idx, val) in enumerate(predicted_with_marriage_assigment) if val==cat_idx];
    docs_in_topic_notin_cat = []
    for i in topic_x_top_docs 
        if !(i in docs_ls[cat_idx])
            push!(docs_in_topic_notin_cat, i)
        end
    end
    return docs_in_topic_notin_cat
end

false_docs_in_cat (generic function with 1 method)

In [31]:
function topic_of_each_doc(lda, corpus)
    top_topic_for_each_doc = []
    for i in 1:corpus.document_size
        top_val = 0
        top_topic = 0
        for j in 1:lda.M
            v = TopicModels.topicPredict(lda, i, j)
            if v>=top_val
                top_val = v
                top_topic = j
            end
        end
        push!(top_topic_for_each_doc, top_topic)
    end
    return top_topic_for_each_doc
end

topic_of_each_doc (generic function with 1 method)

## Remove_kp refinement after stable marriage algorithm (5 times)(repeat 10 times)

In [6]:
true_cat_original = [17, 7, 18, 17, 10, 19, 16, 19, 18, 8, 13, 7, 12, 9, 18, 18, 17, 17, 15, 17, 7, 1, 19, 5, 4, 8, 19, 17, 15, 15, 17, 0, 19, 11, 17, 18, 9, 13, 19, 17, 13, 17, 19, 15, 7, 8, 11, 9, 1, 8, 0, 18, 17, 14, 14, 10, 15, 3, 8, 10, 10, 11, 17, 6, 17, 17, 15, 8, 15, 11, 2, 17, 9, 18, 17, 17, 10, 2, 9, 1, 10, 4, 9, 15, 3, 15, 19, 15, 16, 13, 17, 11, 0, 18, 10, 14, 6, 17, 15, 17, 19, 14, 8, 14, 5, 5, 11, 15, 6, 10, 5, 9, 15, 11, 9, 15, 7, 1, 14, 16, 15, 13, 13, 17, 0, 11, 16, 3, 16, 15, 14, 14, 19, 9, 18, 14, 17, 17, 4, 17, 15, 1, 16, 11, 15, 6, 2, 14, 15, 18, 8, 17, 18, 4, 7, 11, 17, 18, 9, 16, 18, 7, 11, 2, 15, 19, 1, 11, 14, 17, 1, 16, 6, 5, 19, 7, 11, 16, 8, 16, 10, 15, 17, 16, 16, 1, 0, 7, 15, 0, 6, 18, 10, 9, 11, 16, 19, 14, 6, 0, 0, 3, 17, 17, 12, 16, 8, 6, 19, 10, 17, 16, 14, 4, 16, 11, 1, 14, 0, 2, 16, 15, 15, 6, 15, 0, 14, 10, 15, 13, 2, 19, 15, 17, 15, 1, 9, 16, 18, 17, 6, 17, 8, 14, 11, 5, 15, 7, 15, 15, 12, 11, 14, 14, 3, 15, 3, 17, 3, 14, 11, 19, 10, 18, 15, 16, 15, 15, 9, 16, 11, 4, 19, 13, 3, 19, 9, 17, 18, 7, 7, 15, 16, 14, 15, 10, 5, 9, 14, 14, 17, 15, 15, 19, 14, 15, 15, 0, 4, 11, 11, 15, 12, 6, 3, 15, 14, 5, 14, 18, 11, 11, 14, 17, 17, 10, 9, 5, 7, 10, 19, 18, 17, 15, 18, 17, 11, 11, 13, 3, 2, 15, 10, 17, 2, 14, 13, 15, 15, 16, 15, 13, 14, 7, 16, 19, 3, 17, 17, 17, 14, 16, 9, 8, 14, 14, 16, 13, 12, 11, 17, 11, 1, 4, 11, 18, 16, 15, 11, 17, 15, 6, 17, 17, 13, 9, 10, 18, 17, 17, 13, 6, 1, 9, 14, 19, 3, 19, 13, 14, 10, 16, 19, 14, 9, 17, 7, 7, 11, 17, 18, 11, 17, 11, 18, 17, 16, 14, 1, 11, 16, 11, 18, 6, 1, 14, 15, 6, 17, 18, 16, 6, 13, 11, 0, 0, 16, 8, 17, 17, 10, 4, 15, 15, 15, 1, 13, 15, 14, 5, 15, 9, 1, 0, 13, 11, 1, 14, 16, 7, 2, 6, 10, 8, 14, 12, 19, 2, 7, 15, 17, 13, 17, 13, 17, 17, 19, 16, 15, 0, 12, 19, 19, 18, 15, 16, 9, 15, 16, 16, 12, 9, 9, 18, 4, 18, 11, 17, 15, 12, 12, 17, 17, 15, 19, 15, 19, 17, 15, 10, 14, 19, 16, 17, 10, 10, 14, 16, 5, 13, 9, 15, 17, 15, 19, 0, 18, 14, 15, 1, 15, 5, 17, 12, 18, 10, 5, 16, 0, 15, 1, 5, 7, 11, 19, 13, 11, 17, 9, 18, 9, 4, 10, 11, 5, 15, 12, 15, 1, 18, 6, 13, 18, 9, 1, 17, 18, 17, 15, 15, 19, 3, 17, 15, 9, 11, 14, 17, 14, 15, 1, 15, 15, 10, 3, 19, 17, 14, 6, 11, 12, 15, 17, 15, 13, 3, 0, 10, 4, 11, 0, 14, 7, 0, 15, 0, 2, 17, 17, 6, 19, 14, 17, 14, 6, 15, 12, 13, 4, 1, 17, 16, 17, 13, 18, 0, 11, 10, 3, 0, 13, 15, 13, 15, 15, 13, 5, 18, 1, 11, 19, 15, 13, 13, 10, 17, 13, 14, 19, 14, 0, 5, 7, 15, 11, 1, 15, 15, 14, 15, 13, 0, 15, 18, 7, 8, 13, 11, 16, 14, 19, 15, 16, 0, 4, 17, 11, 10, 11, 5, 19, 10, 17, 7, 10, 17, 9, 11, 15, 19, 17, 11, 11, 11, 10, 16, 14, 17, 18, 13, 16, 19, 9, 11, 18, 16, 1, 13, 16, 18, 16, 11, 16, 8, 14, 14, 16, 15, 13, 15, 11, 11, 16, 2, 15, 16, 19, 8, 17, 19, 0, 16, 15, 14, 17, 9, 19, 6, 10, 16, 15, 0, 17, 10, 6, 16, 16, 17, 11, 15, 14, 10, 2, 18, 0, 18, 16, 10, 8, 3, 17, 17, 15, 11, 17, 13, 9, 18, 19, 11, 17, 11, 11, 5, 18, 17, 13, 11, 14, 13, 17, 15, 2, 0, 12, 0, 0, 14, 15, 8, 18, 9, 17, 10, 15, 13, 13, 3, 4, 16, 10, 5, 2, 2, 16, 17, 15, 7, 15, 17, 16, 12, 17, 15, 16, 15, 17, 17, 17, 18, 19, 17, 14, 18, 11, 12, 18, 6, 14, 11, 0, 15, 10, 17, 3, 11, 9, 15, 11, 9, 13, 16, 15, 13, 17, 18, 10, 4, 18, 19, 5, 19, 17, 11, 10, 19, 17, 19, 18, 10, 3, 13, 10, 16, 11, 5, 0, 3, 11, 17, 13, 14, 6, 16, 4, 19, 19, 14, 1, 14, 16, 11, 6, 18, 19, 11, 10, 11, 19, 11, 9, 19, 10, 17, 10, 18, 14, 11, 9, 0, 9, 1, 15, 11, 8, 14, 18, 11, 16, 15, 16, 6, 18, 13, 18, 2, 17, 15, 16, 15, 11, 16, 5, 15, 11, 14, 1, 16, 15, 11, 19, 16, 11, 15, 17, 15, 2, 16, 11, 7, 13, 17, 16, 7, 10, 1, 16, 1, 18, 17, 11, 17, 11, 5, 17, 1, 18, 0, 18, 16, 17, 12, 17, 11, 11, 15, 15, 12, 0, 19, 10, 17, 14, 12, 0, 19, 11, 2, 16, 19, 17, 15, 10, 5, 16, 5, 5, 13, 11, 19, 4, 15, 14, 11, 17, 15, 14, 8, 17, 3, 13, 0, 17, 0, 10, 12, 14, 6, 13, 13, 11, 10, 13, 10, 14, 0, 19, 11, 7, 10, 14, 3, 15, 3, 14, 10, 17, 4, 5, 11, 0, 19, 10, 10, 4, 1, 13, 17, 7, 5, 19, 13, 11, 14, 19, 19, 18, 2, 13, 6, 9, 0, 11, 11, 6, 11, 18, 16, 14, 10, 2, 8, 16, 5, 17, 16, 13, 18, 17, 13, 18, 15, 19, 6, 18, 17, 19, 12, 18, 9, 13, 6, 0, 15, 11, 18, 11, 0, 18, 15, 10, 10, 6, 5, 17, 11, 1, 9, 1, 14, 12, 14, 10, 15, 11, 11, 18, 2, 10, 3, 17, 18, 14, 19, 17, 15, 9, 5, 1, 9, 13, 15, 13, 10, 17, 19, 4, 17, 17, 15, 10, 17, 13, 18, 17, 11, 18, 18, 15, 15, 17, 11, 10, 8, 0, 15, 15, 14, 11, 15, 11, 8, 15, 3, 12, 8, 15, 14, 15, 10, 10, 11, 11, 3, 15, 9, 19, 17, 4, 7, 15, 4, 9, 9, 1, 6, 13, 10, 19, 9, 11, 16, 19, 17, 16, 14, 10, 9, 15, 15, 9, 5, 0, 2, 18, 9, 15, 19, 19, 17, 5, 10, 14, 15, 17, 15, 18, 17, 15, 4, 8, 13, 8, 11, 1, 15, 13, 15, 9, 9, 17, 14, 10, 14, 0, 15, 0, 17, 10, 0, 17, 0, 10, 14, 0, 12, 16, 17, 13, 13, 19, 3, 15, 0, 16, 19, 11, 14, 17, 19, 0, 14, 2, 18, 17, 15, 2, 17, 3, 11, 14, 9, 4, 3, 9, 0, 11, 13, 5, 15, 10, 14, 10, 15, 0, 9, 17, 13, 0, 5, 17, 16, 7, 0, 19, 15, 18, 15, 17, 0, 1, 15, 11, 17, 15, 9, 18, 6, 11, 17, 12, 13, 15, 18, 7, 17, 15, 15, 6, 13, 11, 18, 4, 5, 13, 7, 17, 18, 17, 15, 18, 0, 6, 5, 18, 13, 14, 2, 19, 8, 8, 10, 8, 17, 11, 15, 10, 17, 0, 19, 8, 6, 15, 12, 15, 16, 12, 19, 6, 6, 9, 14, 14, 17, 16, 16, 13, 6, 13, 15, 6, 15, 0, 10, 10, 15, 13, 14, 15, 17, 10, 15, 17, 11, 5, 9, 12, 16, 10, 13, 16, 2, 17, 6, 5, 0, 15, 9, 15, 19, 0, 11, 18, 1, 3, 18, 18, 18, 17, 0, 4, 17, 15, 15, 17, 1, 1, 15, 1, 15, 15, 16, 19, 17, 15, 2, 17, 15, 11, 16, 11, 15, 12, 15, 6, 13, 16, 19, 3, 5, 5, 7, 6, 1, 1, 16, 15, 18, 0, 15, 0, 15, 17, 19, 15, 0, 16, 14, 17, 6, 7, 15, 11, 12, 15, 15, 17, 15, 12, 17, 1, 1, 14, 3, 6, 17, 17, 18, 15, 1, 6, 10, 18, 10, 2, 11, 13, 17, 16, 3, 17, 19, 19, 18, 15, 8, 5, 16, 11, 0, 9, 13, 0, 9, 10, 1, 15, 1, 10, 17, 1, 14, 16, 1, 14, 15, 11, 14, 6, 8, 15, 9, 19, 16, 7, 17, 14, 17, 15, 18, 5, 11, 15, 2, 16, 5, 15, 9, 12, 17, 10, 15, 17, 19, 9, 1, 15, 17, 18, 3, 16, 18, 1, 13, 3, 18, 18, 15, 17, 17, 9, 19, 3, 11, 16, 17, 10, 1, 7, 19, 9, 19, 1, 17, 10, 18, 17, 8, 10, 17, 19, 13, 5, 18, 15, 11, 19, 0, 14, 11, 15, 15, 18, 18, 16, 13, 2, 16, 12, 5, 6, 0, 5, 15, 3, 16, 8, 8, 1, 17, 13, 18, 17, 17, 14, 6, 16, 14, 4, 3, 3, 3, 17, 7, 4, 14, 15, 18, 11, 16, 17, 15, 17, 6, 9, 13, 17, 6, 17, 18, 3, 17, 15, 3, 15, 18, 10, 17, 2, 18, 11, 13, 11, 15, 1, 10, 18, 19, 14, 11, 18, 15, 0, 1, 15, 17, 15, 8, 11, 18, 1, 7, 19, 10, 0, 9, 17, 9, 17, 11, 1, 19, 7, 15, 13, 17, 17, 0, 9, 16, 18, 18, 13, 15, 5, 7, 15, 15, 4, 15, 5, 5, 8, 9, 11, 13, 12, 16, 17, 17, 1, 19, 15, 14, 13, 11, 13, 13, 13, 14, 2, 9, 15, 7, 15, 2, 8, 11, 15, 11, 0, 4, 11, 15, 0, 15, 1, 5, 17, 5, 11, 5, 4, 8, 17, 3, 5, 14, 5, 13, 10, 16, 17, 9, 7, 6, 10, 10, 19, 3, 14, 17, 13, 1, 16, 0, 7, 17, 15, 0, 11, 15, 19, 16, 18, 0, 10, 17, 1, 17, 1, 15, 15, 9, 12, 14, 15, 11, 10, 11, 16, 13, 17, 15, 2, 13, 19, 19, 15, 16, 10, 19, 6, 14, 17, 13, 19, 15, 6, 17, 16, 15, 17, 18, 7, 12, 14, 17, 13, 1, 5, 9, 0, 6, 4, 17, 4, 13, 17, 16, 19, 1, 6, 14, 11, 11, 9, 14, 10, 16, 3, 12, 17, 15, 17, 1, 3, 13, 14, 10, 5, 14, 15, 6, 10, 15, 16, 14, 15, 15, 9, 1, 17, 15, 19, 17, 2, 9, 9, 1, 15, 17, 17, 15, 16, 16, 14, 9, 17, 17, 10, 4, 15, 17, 16, 18, 18, 19, 17, 3, 12, 7, 11, 3, 13, 12, 15, 1, 19, 17, 12, 16, 9, 8, 14, 0, 16, 5, 10, 15, 15, 15, 12, 17, 18, 15, 14, 2, 15, 18, 0, 15, 4, 1, 1, 1, 13, 13, 3, 16, 9, 19, 17, 12, 10, 18, 0, 0, 14, 7, 13, 14, 17, 13, 17, 1, 6, 9, 15, 4, 15, 8, 17, 18, 7, 19, 16, 9, 6, 16, 10, 14, 6, 18, 18, 11, 17, 7, 18, 13, 3, 17, 17, 15, 19, 19, 11, 0, 17, 15, 19, 18, 15, 5, 10, 0, 0, 0, 1, 13, 0, 12, 18, 8, 0, 2, 15, 17, 18, 6, 17, 13, 15, 0, 17, 18, 15, 18, 13, 19, 10, 8, 15, 2, 1, 15, 17, 15, 17, 17]
docs_cat = [i+1 for i in true_cat_original];
docs_ls = [[] for i in 1:20]
for (idx, val) in enumerate(docs_cat)
    push!(docs_ls[val], idx)
end

In [7]:
topic_marriage_assigment = Dict(1=>16, 2=>9, 3=>20, 4=>5, 5=>18, 6=>10, 7=>2, 8=>1, 9=>13, 10=>4, 
                            11=>7, 12=>12, 13=>8, 14=>17, 15=>6, 16=>14, 17=>19, 18=>15, 19=>11, 20=>3);

In [13]:
#First loop refinement for all topics  #run 10 times and average it. 
for current_cat in 1:20
    false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
    if false_docs_for_cat != []
        cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, [false_docs_for_cat], 1, 1, 0.75)
        current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
        TopicModels.removeDoc(lda, corpus, docs_have[1], current_topic)
        TopicModels.gibbsSampling(lda, corpus.documents, 20);
    end
end

In [15]:
#Second loop refinement for all topics
for current_cat in 1:20
    false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
    if false_docs_for_cat != []
        cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, [false_docs_for_cat], 1, 1, 0.75)
        current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
        TopicModels.removeDoc(lda, corpus, docs_have[1], current_topic)
        TopicModels.gibbsSampling(lda, corpus.documents, 20);
    end
end

In [17]:
#Third loop refinement for all topics
for current_cat in 1:20
    false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
    if false_docs_for_cat != []
        cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, [false_docs_for_cat], 1, 1, 0.75)
        current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
        TopicModels.removeDoc(lda, corpus, docs_have[1], current_topic)
        TopicModels.gibbsSampling(lda, corpus.documents, 20);
    end
end

In [164]:
#Fourth loop refinement for all topics
for current_cat in 1:20
    false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
    if false_docs_for_cat != []
        cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, [false_docs_for_cat], 1, 1, 0.75)
        current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
        TopicModels.removeDoc(lda, corpus, docs_have[1], current_topic)
        TopicModels.gibbsSampling(lda, corpus.documents, 20);
    end
end

In [33]:
#Fifth loop refinement for all topics
for current_cat in 1:20
    false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
    if false_docs_for_cat != []
        cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, [false_docs_for_cat], 1, 1, 0.75)
        current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
        TopicModels.removeDoc(lda, corpus, docs_have[1], current_topic)
        TopicModels.gibbsSampling(lda, corpus.documents, 20);
    end
end

In [ ]:
top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
print(predicted_with_marriage_assigment)

In [51]:
# For average F1 score graph repeat each loop 10 times

In [49]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
for loop in 1:5
    for current_cat in 1:20
        false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
        if false_docs_for_cat != []
            cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, [false_docs_for_cat], 1, 1, 0.75)
            current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
            TopicModels.removeDoc(lda, corpus, docs_have[1], current_topic)
            TopicModels.gibbsSampling(lda, corpus.documents, 20);
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
end

0.40715968308965084
0.44761922907736856
0.4884414456072598
0.513950478869657
0.49671982726923436


In [38]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
for loop in 1:5
    for current_cat in 1:20
        false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
        if false_docs_for_cat != []
            cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, [false_docs_for_cat], 1, 1, 0.75)
            current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
            TopicModels.removeDoc(lda, corpus, docs_have[1], current_topic)
            TopicModels.gibbsSampling(lda, corpus.documents, 20);
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
end

0.4263987160163777
0.4493100241246682
0.511025028923098
0.5313629421477779
0.5555996235548682


In [50]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
for loop in 1:5
    for current_cat in 1:20
        false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
        if false_docs_for_cat != []
            cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, [false_docs_for_cat], 1, 1, 0.75)
            current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
            TopicModels.removeDoc(lda, corpus, docs_have[1], current_topic)
            TopicModels.gibbsSampling(lda, corpus.documents, 20);
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
end

0.4231224578434103
0.47888412754799475
0.5137788456242917
0.5452544082318326
0.5924623614018195


In [40]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
for loop in 1:5
    for current_cat in 1:20
        false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
        if false_docs_for_cat != []
            cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, [false_docs_for_cat], 1, 1, 0.75)
            current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
            TopicModels.removeDoc(lda, corpus, docs_have[1], current_topic)
            TopicModels.gibbsSampling(lda, corpus.documents, 20);
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
end

0.43852465426672205
0.4509907859664557
0.41843268932906386
0.4891594007459804
0.522901230649405


In [41]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
for loop in 1:5
    for current_cat in 1:20
        false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
        if false_docs_for_cat != []
            cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, [false_docs_for_cat], 1, 1, 0.75)
            current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
            TopicModels.removeDoc(lda, corpus, docs_have[1], current_topic)
            TopicModels.gibbsSampling(lda, corpus.documents, 20);
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
end

0.4207942190758324
0.48224284937465395
0.4809345988828039
0.5269625042356878
0.5826378599987546


In [42]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
for loop in 1:5
    for current_cat in 1:20
        false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
        if false_docs_for_cat != []
            cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, [false_docs_for_cat], 1, 1, 0.75)
            current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
            TopicModels.removeDoc(lda, corpus, docs_have[1], current_topic)
            TopicModels.gibbsSampling(lda, corpus.documents, 20);
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
end

0.43032779843499996
0.46493379000033375
0.5088458481016407
0.5056242544510859
0.5457140649364104


In [43]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
for loop in 1:5
    for current_cat in 1:20
        false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
        if false_docs_for_cat != []
            cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, [false_docs_for_cat], 1, 1, 0.75)
            current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
            TopicModels.removeDoc(lda, corpus, docs_have[1], current_topic)
            TopicModels.gibbsSampling(lda, corpus.documents, 20);
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
end

0.43610743953734993
0.4950801821633983
0.5108369742202062
0.5480885525274964
0.5313184766082483


In [44]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
for loop in 1:5
    for current_cat in 1:20
        false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
        if false_docs_for_cat != []
            cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, [false_docs_for_cat], 1, 1, 0.75)
            current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
            TopicModels.removeDoc(lda, corpus, docs_have[1], current_topic)
            TopicModels.gibbsSampling(lda, corpus.documents, 20);
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
end

0.4259438702719369
0.4194072658924883
0.4780083201683894
0.4965140624739341
0.5242814320080812


In [45]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
for loop in 1:5
    for current_cat in 1:20
        false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
        if false_docs_for_cat != []
            cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, [false_docs_for_cat], 1, 1, 0.75)
            current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
            TopicModels.removeDoc(lda, corpus, docs_have[1], current_topic)
            TopicModels.gibbsSampling(lda, corpus.documents, 20);
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
end

0.4244259322138922
0.46173970289814315
0.42952228301086526
0.47161589643870994
0.5122862190155076


In [46]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
for loop in 1:5
    for current_cat in 1:20
        false_docs_for_cat = false_docs_in_cat(lda, corpus, docs_ls, current_cat)
        if false_docs_for_cat != []
            cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, [false_docs_for_cat], 1, 1, 0.75)
            current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
            TopicModels.removeDoc(lda, corpus, docs_have[1], current_topic)
            TopicModels.gibbsSampling(lda, corpus.documents, 20);
        end
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    println(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
end

0.4195130305336595
0.43938520975167084
0.4763695276997123
0.546709274049811
0.5513024446892776


In [9]:
function false_docs_in_cat(lda, corpus, docs_ls, cat_idx)
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    topic_x_top_docs = [idx for (idx, val) in enumerate(predicted_with_marriage_assigment) if val==cat_idx];
    docs_in_topic_notin_cat = []
    for i in topic_x_top_docs 
        if !(i in docs_ls[cat_idx])
            push!(docs_in_topic_notin_cat, i)
        end
    end
    return docs_in_topic_notin_cat
end

false_docs_in_cat (generic function with 1 method)

In [10]:
function topic_of_each_doc(lda, corpus)
    top_topic_for_each_doc = []
    for i in 1:corpus.document_size
        top_val = 0
        top_topic = 0
        for j in 1:lda.M
            v = TopicModels.topicPredict(lda, i, j)
            if v>=top_val
                top_val = v
                top_topic = j
            end
        end
        push!(top_topic_for_each_doc, top_topic)
    end
    return top_topic_for_each_doc
end

topic_of_each_doc (generic function with 1 method)

## Add_kp refinement after stable marriage algorithm

In [15]:
true_cat_original = [17, 7, 18, 17, 10, 19, 16, 19, 18, 8, 13, 7, 12, 9, 18, 18, 17, 17, 15, 17, 7, 1, 19, 5, 4, 8, 19, 17, 15, 15, 17, 0, 19, 11, 17, 18, 9, 13, 19, 17, 13, 17, 19, 15, 7, 8, 11, 9, 1, 8, 0, 18, 17, 14, 14, 10, 15, 3, 8, 10, 10, 11, 17, 6, 17, 17, 15, 8, 15, 11, 2, 17, 9, 18, 17, 17, 10, 2, 9, 1, 10, 4, 9, 15, 3, 15, 19, 15, 16, 13, 17, 11, 0, 18, 10, 14, 6, 17, 15, 17, 19, 14, 8, 14, 5, 5, 11, 15, 6, 10, 5, 9, 15, 11, 9, 15, 7, 1, 14, 16, 15, 13, 13, 17, 0, 11, 16, 3, 16, 15, 14, 14, 19, 9, 18, 14, 17, 17, 4, 17, 15, 1, 16, 11, 15, 6, 2, 14, 15, 18, 8, 17, 18, 4, 7, 11, 17, 18, 9, 16, 18, 7, 11, 2, 15, 19, 1, 11, 14, 17, 1, 16, 6, 5, 19, 7, 11, 16, 8, 16, 10, 15, 17, 16, 16, 1, 0, 7, 15, 0, 6, 18, 10, 9, 11, 16, 19, 14, 6, 0, 0, 3, 17, 17, 12, 16, 8, 6, 19, 10, 17, 16, 14, 4, 16, 11, 1, 14, 0, 2, 16, 15, 15, 6, 15, 0, 14, 10, 15, 13, 2, 19, 15, 17, 15, 1, 9, 16, 18, 17, 6, 17, 8, 14, 11, 5, 15, 7, 15, 15, 12, 11, 14, 14, 3, 15, 3, 17, 3, 14, 11, 19, 10, 18, 15, 16, 15, 15, 9, 16, 11, 4, 19, 13, 3, 19, 9, 17, 18, 7, 7, 15, 16, 14, 15, 10, 5, 9, 14, 14, 17, 15, 15, 19, 14, 15, 15, 0, 4, 11, 11, 15, 12, 6, 3, 15, 14, 5, 14, 18, 11, 11, 14, 17, 17, 10, 9, 5, 7, 10, 19, 18, 17, 15, 18, 17, 11, 11, 13, 3, 2, 15, 10, 17, 2, 14, 13, 15, 15, 16, 15, 13, 14, 7, 16, 19, 3, 17, 17, 17, 14, 16, 9, 8, 14, 14, 16, 13, 12, 11, 17, 11, 1, 4, 11, 18, 16, 15, 11, 17, 15, 6, 17, 17, 13, 9, 10, 18, 17, 17, 13, 6, 1, 9, 14, 19, 3, 19, 13, 14, 10, 16, 19, 14, 9, 17, 7, 7, 11, 17, 18, 11, 17, 11, 18, 17, 16, 14, 1, 11, 16, 11, 18, 6, 1, 14, 15, 6, 17, 18, 16, 6, 13, 11, 0, 0, 16, 8, 17, 17, 10, 4, 15, 15, 15, 1, 13, 15, 14, 5, 15, 9, 1, 0, 13, 11, 1, 14, 16, 7, 2, 6, 10, 8, 14, 12, 19, 2, 7, 15, 17, 13, 17, 13, 17, 17, 19, 16, 15, 0, 12, 19, 19, 18, 15, 16, 9, 15, 16, 16, 12, 9, 9, 18, 4, 18, 11, 17, 15, 12, 12, 17, 17, 15, 19, 15, 19, 17, 15, 10, 14, 19, 16, 17, 10, 10, 14, 16, 5, 13, 9, 15, 17, 15, 19, 0, 18, 14, 15, 1, 15, 5, 17, 12, 18, 10, 5, 16, 0, 15, 1, 5, 7, 11, 19, 13, 11, 17, 9, 18, 9, 4, 10, 11, 5, 15, 12, 15, 1, 18, 6, 13, 18, 9, 1, 17, 18, 17, 15, 15, 19, 3, 17, 15, 9, 11, 14, 17, 14, 15, 1, 15, 15, 10, 3, 19, 17, 14, 6, 11, 12, 15, 17, 15, 13, 3, 0, 10, 4, 11, 0, 14, 7, 0, 15, 0, 2, 17, 17, 6, 19, 14, 17, 14, 6, 15, 12, 13, 4, 1, 17, 16, 17, 13, 18, 0, 11, 10, 3, 0, 13, 15, 13, 15, 15, 13, 5, 18, 1, 11, 19, 15, 13, 13, 10, 17, 13, 14, 19, 14, 0, 5, 7, 15, 11, 1, 15, 15, 14, 15, 13, 0, 15, 18, 7, 8, 13, 11, 16, 14, 19, 15, 16, 0, 4, 17, 11, 10, 11, 5, 19, 10, 17, 7, 10, 17, 9, 11, 15, 19, 17, 11, 11, 11, 10, 16, 14, 17, 18, 13, 16, 19, 9, 11, 18, 16, 1, 13, 16, 18, 16, 11, 16, 8, 14, 14, 16, 15, 13, 15, 11, 11, 16, 2, 15, 16, 19, 8, 17, 19, 0, 16, 15, 14, 17, 9, 19, 6, 10, 16, 15, 0, 17, 10, 6, 16, 16, 17, 11, 15, 14, 10, 2, 18, 0, 18, 16, 10, 8, 3, 17, 17, 15, 11, 17, 13, 9, 18, 19, 11, 17, 11, 11, 5, 18, 17, 13, 11, 14, 13, 17, 15, 2, 0, 12, 0, 0, 14, 15, 8, 18, 9, 17, 10, 15, 13, 13, 3, 4, 16, 10, 5, 2, 2, 16, 17, 15, 7, 15, 17, 16, 12, 17, 15, 16, 15, 17, 17, 17, 18, 19, 17, 14, 18, 11, 12, 18, 6, 14, 11, 0, 15, 10, 17, 3, 11, 9, 15, 11, 9, 13, 16, 15, 13, 17, 18, 10, 4, 18, 19, 5, 19, 17, 11, 10, 19, 17, 19, 18, 10, 3, 13, 10, 16, 11, 5, 0, 3, 11, 17, 13, 14, 6, 16, 4, 19, 19, 14, 1, 14, 16, 11, 6, 18, 19, 11, 10, 11, 19, 11, 9, 19, 10, 17, 10, 18, 14, 11, 9, 0, 9, 1, 15, 11, 8, 14, 18, 11, 16, 15, 16, 6, 18, 13, 18, 2, 17, 15, 16, 15, 11, 16, 5, 15, 11, 14, 1, 16, 15, 11, 19, 16, 11, 15, 17, 15, 2, 16, 11, 7, 13, 17, 16, 7, 10, 1, 16, 1, 18, 17, 11, 17, 11, 5, 17, 1, 18, 0, 18, 16, 17, 12, 17, 11, 11, 15, 15, 12, 0, 19, 10, 17, 14, 12, 0, 19, 11, 2, 16, 19, 17, 15, 10, 5, 16, 5, 5, 13, 11, 19, 4, 15, 14, 11, 17, 15, 14, 8, 17, 3, 13, 0, 17, 0, 10, 12, 14, 6, 13, 13, 11, 10, 13, 10, 14, 0, 19, 11, 7, 10, 14, 3, 15, 3, 14, 10, 17, 4, 5, 11, 0, 19, 10, 10, 4, 1, 13, 17, 7, 5, 19, 13, 11, 14, 19, 19, 18, 2, 13, 6, 9, 0, 11, 11, 6, 11, 18, 16, 14, 10, 2, 8, 16, 5, 17, 16, 13, 18, 17, 13, 18, 15, 19, 6, 18, 17, 19, 12, 18, 9, 13, 6, 0, 15, 11, 18, 11, 0, 18, 15, 10, 10, 6, 5, 17, 11, 1, 9, 1, 14, 12, 14, 10, 15, 11, 11, 18, 2, 10, 3, 17, 18, 14, 19, 17, 15, 9, 5, 1, 9, 13, 15, 13, 10, 17, 19, 4, 17, 17, 15, 10, 17, 13, 18, 17, 11, 18, 18, 15, 15, 17, 11, 10, 8, 0, 15, 15, 14, 11, 15, 11, 8, 15, 3, 12, 8, 15, 14, 15, 10, 10, 11, 11, 3, 15, 9, 19, 17, 4, 7, 15, 4, 9, 9, 1, 6, 13, 10, 19, 9, 11, 16, 19, 17, 16, 14, 10, 9, 15, 15, 9, 5, 0, 2, 18, 9, 15, 19, 19, 17, 5, 10, 14, 15, 17, 15, 18, 17, 15, 4, 8, 13, 8, 11, 1, 15, 13, 15, 9, 9, 17, 14, 10, 14, 0, 15, 0, 17, 10, 0, 17, 0, 10, 14, 0, 12, 16, 17, 13, 13, 19, 3, 15, 0, 16, 19, 11, 14, 17, 19, 0, 14, 2, 18, 17, 15, 2, 17, 3, 11, 14, 9, 4, 3, 9, 0, 11, 13, 5, 15, 10, 14, 10, 15, 0, 9, 17, 13, 0, 5, 17, 16, 7, 0, 19, 15, 18, 15, 17, 0, 1, 15, 11, 17, 15, 9, 18, 6, 11, 17, 12, 13, 15, 18, 7, 17, 15, 15, 6, 13, 11, 18, 4, 5, 13, 7, 17, 18, 17, 15, 18, 0, 6, 5, 18, 13, 14, 2, 19, 8, 8, 10, 8, 17, 11, 15, 10, 17, 0, 19, 8, 6, 15, 12, 15, 16, 12, 19, 6, 6, 9, 14, 14, 17, 16, 16, 13, 6, 13, 15, 6, 15, 0, 10, 10, 15, 13, 14, 15, 17, 10, 15, 17, 11, 5, 9, 12, 16, 10, 13, 16, 2, 17, 6, 5, 0, 15, 9, 15, 19, 0, 11, 18, 1, 3, 18, 18, 18, 17, 0, 4, 17, 15, 15, 17, 1, 1, 15, 1, 15, 15, 16, 19, 17, 15, 2, 17, 15, 11, 16, 11, 15, 12, 15, 6, 13, 16, 19, 3, 5, 5, 7, 6, 1, 1, 16, 15, 18, 0, 15, 0, 15, 17, 19, 15, 0, 16, 14, 17, 6, 7, 15, 11, 12, 15, 15, 17, 15, 12, 17, 1, 1, 14, 3, 6, 17, 17, 18, 15, 1, 6, 10, 18, 10, 2, 11, 13, 17, 16, 3, 17, 19, 19, 18, 15, 8, 5, 16, 11, 0, 9, 13, 0, 9, 10, 1, 15, 1, 10, 17, 1, 14, 16, 1, 14, 15, 11, 14, 6, 8, 15, 9, 19, 16, 7, 17, 14, 17, 15, 18, 5, 11, 15, 2, 16, 5, 15, 9, 12, 17, 10, 15, 17, 19, 9, 1, 15, 17, 18, 3, 16, 18, 1, 13, 3, 18, 18, 15, 17, 17, 9, 19, 3, 11, 16, 17, 10, 1, 7, 19, 9, 19, 1, 17, 10, 18, 17, 8, 10, 17, 19, 13, 5, 18, 15, 11, 19, 0, 14, 11, 15, 15, 18, 18, 16, 13, 2, 16, 12, 5, 6, 0, 5, 15, 3, 16, 8, 8, 1, 17, 13, 18, 17, 17, 14, 6, 16, 14, 4, 3, 3, 3, 17, 7, 4, 14, 15, 18, 11, 16, 17, 15, 17, 6, 9, 13, 17, 6, 17, 18, 3, 17, 15, 3, 15, 18, 10, 17, 2, 18, 11, 13, 11, 15, 1, 10, 18, 19, 14, 11, 18, 15, 0, 1, 15, 17, 15, 8, 11, 18, 1, 7, 19, 10, 0, 9, 17, 9, 17, 11, 1, 19, 7, 15, 13, 17, 17, 0, 9, 16, 18, 18, 13, 15, 5, 7, 15, 15, 4, 15, 5, 5, 8, 9, 11, 13, 12, 16, 17, 17, 1, 19, 15, 14, 13, 11, 13, 13, 13, 14, 2, 9, 15, 7, 15, 2, 8, 11, 15, 11, 0, 4, 11, 15, 0, 15, 1, 5, 17, 5, 11, 5, 4, 8, 17, 3, 5, 14, 5, 13, 10, 16, 17, 9, 7, 6, 10, 10, 19, 3, 14, 17, 13, 1, 16, 0, 7, 17, 15, 0, 11, 15, 19, 16, 18, 0, 10, 17, 1, 17, 1, 15, 15, 9, 12, 14, 15, 11, 10, 11, 16, 13, 17, 15, 2, 13, 19, 19, 15, 16, 10, 19, 6, 14, 17, 13, 19, 15, 6, 17, 16, 15, 17, 18, 7, 12, 14, 17, 13, 1, 5, 9, 0, 6, 4, 17, 4, 13, 17, 16, 19, 1, 6, 14, 11, 11, 9, 14, 10, 16, 3, 12, 17, 15, 17, 1, 3, 13, 14, 10, 5, 14, 15, 6, 10, 15, 16, 14, 15, 15, 9, 1, 17, 15, 19, 17, 2, 9, 9, 1, 15, 17, 17, 15, 16, 16, 14, 9, 17, 17, 10, 4, 15, 17, 16, 18, 18, 19, 17, 3, 12, 7, 11, 3, 13, 12, 15, 1, 19, 17, 12, 16, 9, 8, 14, 0, 16, 5, 10, 15, 15, 15, 12, 17, 18, 15, 14, 2, 15, 18, 0, 15, 4, 1, 1, 1, 13, 13, 3, 16, 9, 19, 17, 12, 10, 18, 0, 0, 14, 7, 13, 14, 17, 13, 17, 1, 6, 9, 15, 4, 15, 8, 17, 18, 7, 19, 16, 9, 6, 16, 10, 14, 6, 18, 18, 11, 17, 7, 18, 13, 3, 17, 17, 15, 19, 19, 11, 0, 17, 15, 19, 18, 15, 5, 10, 0, 0, 0, 1, 13, 0, 12, 18, 8, 0, 2, 15, 17, 18, 6, 17, 13, 15, 0, 17, 18, 15, 18, 13, 19, 10, 8, 15, 2, 1, 15, 17, 15, 17, 17]
docs_cat = [i+1 for i in true_cat_original];
docs_ls = [[] for i in 1:20]
for (idx, val) in enumerate(docs_cat)
    push!(docs_ls[val], idx)
end

In [16]:
topic_marriage_assigment = Dict(1=>16, 2=>9, 3=>20, 4=>5, 5=>18, 6=>10, 7=>2, 8=>1, 9=>13, 10=>4, 
                            11=>7, 12=>12, 13=>8, 14=>17, 15=>6, 16=>14, 17=>19, 18=>15, 19=>11, 20=>3);

In [142]:
current_cat_topic = 1
absent_docs = docs_not_in_cat(lda, corpus, current_cat_topic);
cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, [absent_docs], 1, 1, 0.85)
println(cluster_kp[1])
println(docs_have[1])

Dict{Any,Any}("archive" => Any["archive", "archive -"])
Any[516, 1135, 1715, 1746]


In [150]:
#First loop refinement for all topics
for current_cat in 1:20
    absent_docs = docs_not_in_cat(lda, corpus, current_cat)
    cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, [absent_docs], 1, 1, 0.85)
    current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
    TopicModels.addDoc(lda, corpus, docs_have[1], current_topic)
    TopicModels.gibbsSampling(lda, corpus.documents, 20);
end

In [153]:
#Second loop refinement for all topics
for current_cat in 1:20
    absent_docs = docs_not_in_cat(lda, corpus, current_cat)
    cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, [absent_docs], 1, 1, 0.85)
    current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
    TopicModels.addDoc(lda, corpus, docs_have[1], current_topic)
    TopicModels.gibbsSampling(lda, corpus.documents, 20);
end

In [157]:
#Third loop refinement for all topics
for current_cat in 1:20
    absent_docs = docs_not_in_cat(lda, corpus, current_cat)
    cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, [absent_docs], 1, 1, 0.85)
    current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
    TopicModels.addDoc(lda, corpus, docs_have[1], current_topic)
    TopicModels.gibbsSampling(lda, corpus.documents, 20);
end

In [159]:
#Forth loop refinement for all topics
for current_cat in 1:20
    absent_docs = docs_not_in_cat(lda, corpus, current_cat)
    cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, [absent_docs], 1, 1, 0.85)
    current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
    TopicModels.addDoc(lda, corpus, docs_have[1], current_topic)
    TopicModels.gibbsSampling(lda, corpus.documents, 20);
end

In [161]:
#Fifth loop refinement for all topics
for current_cat in 1:20
    absent_docs = docs_not_in_cat(lda, corpus, current_cat)
    cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, [absent_docs], 1, 1, 0.85)
    current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
    TopicModels.addDoc(lda, corpus, docs_have[1], current_topic)
    TopicModels.gibbsSampling(lda, corpus.documents, 20);
end

In [ ]:
top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
print(predicted_with_marriage_assigment)

In [163]:
TopicModels.saveLDA(lda, "20news/lda_obj_2000_long_after_5_add_kp_refinement_loops.json")

9136472

In [17]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
@time for loop in 1:5
    for current_cat in 1:20
        absent_docs = docs_not_in_cat(lda, corpus, current_cat)
        cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, [absent_docs], 1, 1, 0.85)
        current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
        TopicModels.addDoc(lda, corpus, docs_have[1], current_topic)
        TopicModels.gibbsSampling(lda, corpus.documents, 20);
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    print(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
    print(", ")
end

0.4981237570161637, 0.5536430887830217, 0.5985042331563979, 0.6301116369046398, 0.6571118709406457, 12337.312139 seconds (174.58 G allocations: 2.925 TiB, 7.83% gc time)


In [18]:
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
@time for loop in 1:5
    for current_cat in 1:20
        absent_docs = docs_not_in_cat(lda, corpus, current_cat)
        cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, [absent_docs], 1, 1, 0.85)
        current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
        TopicModels.addDoc(lda, corpus, docs_have[1], current_topic)
        TopicModels.gibbsSampling(lda, corpus.documents, 20);
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    print(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
    print(", ")
end

0.47834983352410504, 0.5414053178970297, 0.5975092423837479, 0.6375182183336289, 0.6590115982688469, 11541.739819 seconds (174.56 G allocations: 2.925 TiB, 7.25% gc time)


In [19]:
#4th time experiment
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
@time for loop in 1:5
    for current_cat in 1:20
        absent_docs = docs_not_in_cat(lda, corpus, current_cat)
        cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, [absent_docs], 1, 1, 0.85)
        current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
        TopicModels.addDoc(lda, corpus, docs_have[1], current_topic)
        TopicModels.gibbsSampling(lda, corpus.documents, 20);
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    print(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
    print(", ")
end

0.4922246480815575, 0.5599986099243948, 0.6055545087666807, 0.6358811797052127, 0.6551872718799856, 11240.933803 seconds (174.55 G allocations: 2.925 TiB, 9.22% gc time)


In [20]:
#5th time experiment
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
@time for loop in 1:5
    for current_cat in 1:20
        absent_docs = docs_not_in_cat(lda, corpus, current_cat)
        cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, [absent_docs], 1, 1, 0.85)
        current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
        TopicModels.addDoc(lda, corpus, docs_have[1], current_topic)
        TopicModels.gibbsSampling(lda, corpus.documents, 20);
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    print(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
    print(", ")
end

0.48090449130559193, 0.5467769322172225, 0.5924679455294509, 0.6355353801219835, 0.6606495302383468, 11088.489347 seconds (174.54 G allocations: 2.925 TiB, 7.35% gc time)


In [21]:
#6th time experiment
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
@time for loop in 1:5
    for current_cat in 1:20
        absent_docs = docs_not_in_cat(lda, corpus, current_cat)
        cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, [absent_docs], 1, 1, 0.85)
        current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
        TopicModels.addDoc(lda, corpus, docs_have[1], current_topic)
        TopicModels.gibbsSampling(lda, corpus.documents, 20);
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    print(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
    print(", ")
end

0.48852304566623933, 0.5391349963006075, 0.5908350123495628, 0.634459113680114, 0.6574371860681041, 11396.217523 seconds (174.56 G allocations: 2.925 TiB, 8.98% gc time)


In [22]:
#7th time experiment
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
@time for loop in 1:5
    for current_cat in 1:20
        absent_docs = docs_not_in_cat(lda, corpus, current_cat)
        cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, [absent_docs], 1, 1, 0.85)
        current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
        TopicModels.addDoc(lda, corpus, docs_have[1], current_topic)
        TopicModels.gibbsSampling(lda, corpus.documents, 20);
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    print(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
    print(", ")
end

0.4922783803339358, 0.5500721087433691, 0.5977003916121185, 0.6325027349419882, 0.6667342170447722, 11181.287427 seconds (174.56 G allocations: 2.925 TiB, 7.97% gc time)


In [23]:
#8th time experiment
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
@time for loop in 1:5
    for current_cat in 1:20
        absent_docs = docs_not_in_cat(lda, corpus, current_cat)
        cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, [absent_docs], 1, 1, 0.85)
        current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
        TopicModels.addDoc(lda, corpus, docs_have[1], current_topic)
        TopicModels.gibbsSampling(lda, corpus.documents, 20);
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    print(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
    print(", ")
end

0.488235607118574, 0.547561140374842, 0.5898806841442585, 0.6188663372609233, 0.6568470427448289, 11228.159918 seconds (174.55 G allocations: 2.925 TiB, 9.00% gc time)


In [24]:
#9th time experiment
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
@time for loop in 1:5
    for current_cat in 1:20
        absent_docs = docs_not_in_cat(lda, corpus, current_cat)
        cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, [absent_docs], 1, 1, 0.85)
        current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
        TopicModels.addDoc(lda, corpus, docs_have[1], current_topic)
        TopicModels.gibbsSampling(lda, corpus.documents, 20);
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    print(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
    print(", ")
end

0.48149048948335227, 0.5511192665250314, 0.592911515492085, 0.6352784321886389, 0.6601727293092041, 11204.808255 seconds (174.55 G allocations: 2.925 TiB, 8.12% gc time)


In [25]:
#10th time experiment
lda = TopicModels.loadLDA("20news/lda_obj_2000_long.json");
@time for loop in 1:5
    for current_cat in 1:20
        absent_docs = docs_not_in_cat(lda, corpus, current_cat)
        cluster_kp, docs_have = TopicModels.top_x_kp_of_topic_m(kp, [absent_docs], 1, 1, 0.85)
        current_topic = [k for (k,v) in topic_marriage_assigment if v==current_cat][1]
        TopicModels.addDoc(lda, corpus, docs_have[1], current_topic)
        TopicModels.gibbsSampling(lda, corpus.documents, 20);
    end
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    print(f1_score.f1_score(docs_cat, predicted_with_marriage_assigment, average="weighted"))
    print(", ")
end

0.486556244110898, 0.555697978115983, 0.6093576275552786, 0.6224150907475438, 0.6558080317371272, 11282.479334 seconds (174.54 G allocations: 2.925 TiB, 9.05% gc time)


In [8]:
function docs_not_in_cat(lda, corpus, cat_idx)
    top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
    predicted_with_marriage_assigment = [topic_marriage_assigment[i] for i in top_topic_for_each_doc]
    topic_x_top_docs = [idx for (idx, val) in enumerate(predicted_with_marriage_assigment) if val==cat_idx];
    docs_in_cat_notin_topic = []
    for i in docs_ls[cat_idx]
        if !(i in topic_x_top_docs)
            push!(docs_in_cat_notin_topic, i)
        end
    end
    return docs_in_cat_notin_topic
end

docs_not_in_cat (generic function with 1 method)

In [7]:
function top_kp_from_these_docs(these_docs)
    return TopicModels.keyphrases_of_topic(kp, [these_docs], 1)
end

top_kp_from_these_docs (generic function with 1 method)

In [6]:
function topic_of_each_doc(lda, corpus)
    top_topic_for_each_doc = []
    for i in 1:corpus.document_size
        top_val = 0
        top_topic = 0
        for j in 1:lda.M
            v = TopicModels.topicPredict(lda, i, j)
            if v>=top_val
                top_val = v
                top_topic = j
            end
        end
        push!(top_topic_for_each_doc, top_topic)
    end
    return top_topic_for_each_doc
end

topic_of_each_doc (generic function with 1 method)

## 3 refinement to improve cat christian with topic 1 association

In [12]:
cluster_kp_8, docs_have_8 = TopicModels.top_x_kp_of_topic_m(kp, topic_distributions, 10, 8, 0.75);

In [14]:
x = 8
println(sort(cluster_kp_8[x]))
print(sort(docs_have_8[x]))

OrderedDict{Any,Any}("god" => Any["bible", "god", "christ", "jesus", "scripture", "no god\" and experiences\"", "goodness"])
Any[32, 444, 584, 644, 986, 992, 1129, 1335, 1513, 1542, 1607, 1803]

In [ ]:
#Choose which one is best (for now subjective analysis)(choosing kp_8)
topic_8_kp_1 -> 8 docs out of 29 of cat "christ"
topic_8_kp_2 -> 4 docs out of 9 of cat "christ"
topic_8_kp_3 -> 0 docs out of 7 of cat "christ"
topic_8_kp_4 -> 7 docs out of 22 of cat "christ"
topic_8_kp_5 -> 3 docs out of 10 of cat "christ"
topic_8_kp_6 -> 3 docs out of 6 of cat "christ"
topic_8_kp_7 -> 3 docs out of 7 of cat "christ"
topic_8_kp_8 -> 7 docs out of 12 of cat "christ"
topic_8_kp_9 -> 6 docs out of 13 of cat "christ"
topic_8_kp_10 -> 3 docs out of 4 of cat "christ"

In [27]:
#Apply refinement of add topic8_kp_8 to topic1
TopicModels.addDoc(lda, corpus, docs_have_8[8], 1)
TopicModels.gibbsSampling(lda, corpus.documents, 20);

In [37]:
top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
#println(top_topic_for_each_doc)

In [38]:
topic_distributions = TopicModels.sortedTopDocsForTopics(lda, corpus);
cluster_kp_8, docs_have_8 = TopicModels.top_x_kp_of_topic_m(kp, topic_distributions, 10, 8, 0.75);

In [52]:
x = 1
println(sort(cluster_kp_8[x]))
print(sort(docs_have_8[x]))

OrderedDict{Any,Any}("religion" => Any["bible", "morality", "moral", "religion", "atheism", "sin", "atheism?", "spiritual", "culture", "atheist", "christianity", "theism", "\"judaism"])
Any[233, 262, 457, 587, 620, 745, 908, 959, 965, 1021, 1042, 1094, 1217, 1256, 1405, 1436, 1446, 1571, 1762, 1778, 1782, 1793, 1901, 1918, 1958, 1966, 1984, 1999]

In [ ]:
#Choose which one is best (for now subjective analysis)(choosing kp_1)
topic_8_kp_1 -> 7 docs out of 28 of cat "christ"
topic_8_kp_2 -> 4 docs out of 11 of cat "christ"
topic_8_kp_3 -> 5 docs out of 21 of cat "christ"
topic_8_kp_4 -> 1 docs out of 7 of cat "christ"
topic_8_kp_5 -> 3 docs out of 12 of cat "christ"
topic_8_kp_6 -> 3 docs out of 8 of cat "christ"
topic_8_kp_7 -> 2 docs out of 5 of cat "christ"
topic_8_kp_8 -> 10 docs out of 37 of cat "christ"
topic_8_kp_9 -> 2 docs out of 4 of cat "christ"
topic_8_kp_10 -> 0 docs out of 15 of cat "christ"


In [49]:
#Apply refinement of add topic8_kp_1 to topic1
TopicModels.addDoc(lda, corpus, docs_have_8[1], 1)
TopicModels.gibbsSampling(lda, corpus.documents, 20);

In [51]:
top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
#println(top_topic_for_each_doc)

In [53]:
topic_distributions = TopicModels.sortedTopDocsForTopics(lda, corpus);
cluster_kp_8, docs_have_8 = TopicModels.top_x_kp_of_topic_m(kp, topic_distributions, 10, 8, 0.75);

In [63]:
x = 10
println(sort(cluster_kp_8[x]))
print(sort(docs_have_8[x]))

OrderedDict{Any,Any}("argument" => Any["reasoning", "argument", "interpretation", "contention", "rationalism", "discussion"])
Any[530, 573, 582, 595, 669, 742, 1003, 1073, 1179, 1459, 1708]

In [ ]:
#Choose which one is best (for now subjective analysis)(choosing kp_1)
topic_8_kp_1 -> 6 docs out of 15 of cat "christ"
topic_8_kp_2 -> 4 docs out of 11 of cat "christ"
topic_8_kp_3 -> 0 docs out of 9 of cat "christ"
topic_8_kp_4 -> 5 docs out of 9 of cat "christ"
topic_8_kp_5 -> 5 docs out of 13 of cat "christ"
topic_8_kp_6 -> 3 docs out of 6 of cat "christ"
topic_8_kp_7 -> 2 docs out of 6 of cat "christ"
topic_8_kp_8 -> 1 docs out of 7 of cat "christ"
topic_8_kp_9 -> 1 docs out of 5 of cat "christ"
topic_8_kp_10 -> 6 docs out of 11 of cat "christ"

In [64]:
#Apply refinement of add topic8_kp_10 to topic1
TopicModels.addDoc(lda, corpus, docs_have_8[10], 1)
TopicModels.gibbsSampling(lda, corpus.documents, 20);

In [66]:
top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
#println(top_topic_for_each_doc)

In [ ]:
topic_distributions = TopicModels.sortedTopDocsForTopics(lda, corpus);
cluster_kp_8, docs_have_8 = TopicModels.top_x_kp_of_topic_m(kp, topic_distributions, 10, 8, 0.75);

In [79]:
#Apply refinement of add topic8_kp_4 to topic1
TopicModels.addDoc(lda, corpus, docs_have_8[4], 1)
TopicModels.gibbsSampling(lda, corpus.documents, 20);

In [81]:
top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
#println(top_topic_for_each_doc)

In [82]:
#Apply refinement of add topic8_kp_8 to topic1
TopicModels.addDoc(lda, corpus, docs_have_8[8], 1)
TopicModels.gibbsSampling(lda, corpus.documents, 20);

In [84]:
top_topic_for_each_doc = topic_of_each_doc(lda, corpus);
#println(top_topic_for_each_doc)